### Module Imports 

In [48]:
import os 
import sys
sys.path.insert(0, "//home//nlpsomnath//NegNews//rebase//NN_Kafka//")
os.chdir("//home//nlpsomnath//NegNews//rebase//NN_Kafka//")

import pymagnitude
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import regexp_tokenize
from nltk.tokenize import word_tokenize , sent_tokenize

from gensim import corpora, models, similarities
import string
import re
from fuzzywuzzy import fuzz
import numpy as np
import operator

import pandas as pd
from sklearn import preprocessing 
from scipy import spatial
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from numpy import tanh, exp
from bs4 import BeautifulSoup
from urllib.parse import urlparse




import metadata_parser

from kafka import KafkaProducer, KafkaConsumer, TopicPartition
from multiprocessing import Process
from scrapyd_api import ScrapydAPI
from datetime import datetime
import json
import nn_config as config 
import datetime
#mongo db 
import pymongo
import pandas as pd
from pymongo import MongoClient
import json
import time
import spacy
import dateparser

from nn_base.load_source_data import sqlpulldata
from nn_postprocessing import postprocess
from nn_schedulespiders import schedule_spiders, entity_jobs_finished
from nn_base.nn_dateextractor import date_extract , waybackRes, date_extraction,validate_date
from nn_base.nn_extractfines import get_data_from_article_title, load_entity_dict,transform_article_title
from nn_base.nn_textprocess import url_fuzzy_dupes,VADERSENT ,summary_scores, clean_paragraph, RegVsUnreg, entity_found, entity_found2,trim_key,stemsims,MinHash,namesplit,entity_match_score, entity_stem_match,OrigTxt_PreProcess_DTCC

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

ENTITY_FUZZ_SCORE_THRESHOLD = 90 #fuzzy match threshold for entity count 
FINE_AMOUNT_IDX = 1 #fine amount argument in tuple used to extract fines
FINE_GENERATE_FLAG_IDX = 2 #flag argument in tuples used to extract fines 
known_alias = ['Mizuho']

entity_dict = load_entity_dict("/home/nlpsomnath/.local/lib/python3.5/site-packages/negative_news/resources/entity_dict.pickle")
nlp = spacy.load('en_core_web_sm')
wv=pymagnitude.Magnitude('//home//nlpsomnath//NegNews//rebase//NN_Kafka//sourcefiles//GoogleNews-vectors-negative300.magnitude')

#### 1.Load Source Data from Spider Runs(Stored in Mongo)

In [8]:
#mongo client 
client = MongoClient(waitQueueTimeoutMS=100)

#access data om a database(2 ways shown below)
db_in = client['negative_news']

#list all collections in each database (2 ways)
db_collections = [collection for collection in db_in.collection_names()]
source_data = pd.DataFrame()
source_list=[]
print(db_collections)

#concatenate all collections into a list of dataframes, and aggregate the list of datafraemes into a single large dataframe
source_data = pd.concat([pd.DataFrame(list(db_in[collection].find({}))) for collection in db_collections])
source_data.sort_values(['entity','jobname'], ascending = False, kind = 'mergesort', inplace = True)
source_data = source_data[source_data['entity']!='みずほ']
print('source data dimensions: {}*{}'.format(source_data.shape[0],source_data.shape[1]))

['bing_regulatory_search', 'regulatory_news_search', 'newsapi_search', 'bing_newssearch', 'bing_search']
source data dimensions: 161124*12


In [55]:
input_df = source_data
print(input_df.shape)

#regex patterns to apply to jobname 
replace_values = {'jobname':{r'_':'',r' ':''}}
input_df.replace(replace_values, regex=True, inplace=True)
replace_values = {'Person':{r'_':''}}
input_df.replace(replace_values, regex=True, inplace=True)
input_df['url']=input_df['url'].map(lambda x: x.strip())
df_rlt=pd.DataFrame(input_df)

columnsTitles = ['url','source', 'date', 'entity','content','summary','title','origtext','jobname']
df_rlt=df_rlt.reindex(columns=columnsTitles,copy=True)
df_rlt.rename(columns={'summary': 'Summarylist'}, inplace=True)

print('1.Size of Data at Import:',df_rlt.shape)
#print('Total Sources at time of Import:',df_rlt.source.value_counts())
#print('Total Entities at time of Import:',df_rlt.entity.value_counts())

#get top 2 runids for each entity 
top2jobs = df_rlt[['entity','jobname']].groupby(['entity','jobname'])
dfgroup = [name for name,group in top2jobs]
df2 = pd.DataFrame(dfgroup).drop_duplicates()
df2.rename(index=str, columns={0:"entity", 1: "jobname"}, inplace=True)
df2= df2.sort_values(by=['jobname'],ascending = False)
df3=df2[df2['entity'].apply(lambda x: len(x)>2) & df2['jobname'].apply(lambda x: len(x)>2)]
#top2= df3.groupby('entity').head(1)['jobname']
top2= df3.groupby('entity').head(44)

#df_rlt=df_rlt[df_rlt['jobname'].isin(top2)]

#subset for 
#Bank of America:12-13-18-17-10-46
#Mizuho Bank:12-13-18-17-10-46
#Wells Fargo: 1234948
df_rlt=df_rlt[df_rlt['jobname'].isin(['12-14-18-16-03-26'])]
df_rlt=df_rlt[df_rlt['entity'].isin(['Mizuho Bank'])]
print(top2.head(44))
print(df_rlt.shape)
#print(df_rlt.url.value_counts())

(161124, 12)
1.Size of Data at Import: (161124, 9)
                entity            jobname
1185  Wells Fargo       314              
1184  Wells Fargo       1234948          
444   Deutsche Bank     12-14-18-18-30-58
661   HSBC              12-14-18-18-30-58
443   Deutsche Bank     12-14-18-18-25-42
1203  YMCA              12-14-18-16-14-42
579   GoodWill          12-14-18-16-14-42
666   HSBC Bank         12-14-18-16-03-26
109   Bank of America   12-14-18-16-03-26
503   Fifth Third Bank  12-14-18-16-03-26
320   CitiGroup         12-14-18-16-03-26
442   Deutsche Bank     12-14-18-16-03-26
802   Mizuho Bank       12-14-18-16-03-26
1183  Wells Fargo       12-14-18-16-03-26
319   CitiGroup         12-14-18-15-14-57
801   Mizuho Bank       12-14-18-15-14-57
665   HSBC Bank         12-14-18-15-14-57
1182  Wells Fargo       12-14-18-15-14-57
502   Fifth Third Bank  12-14-18-15-14-57
108   Bank of America   12-14-18-15-14-57
441   Deutsche Bank     12-14-18-15-14-57
1181  Wells Fargo       1

In [57]:
df_step1 = df_preprocess(df_rlt,wv,1)
#df_step1[['url','title']].groupby(['url']).agg([ 'count'])
#df_rlt.entity.unique()


1.Title parsed (162, 11)
2.Size of Data After Empty Content/Summary Removed: (161, 10)
Total Sources at end of URL based filtering and Content/Summary Filter: bingnews                            72
bing                                50
Reuters                             38
Federal Reserve (Press Releases)    1 
Name: source, dtype: int64


In [61]:
df_step2 = enrich_date(df_step1,2018,12)
#df_step2[['url','title','date2']]

Total Sources at time of DateTEST: bingnews                            72
bing                                50
Reuters                             38
Federal Reserve (Press Releases)    1 
Name: source, dtype: int64
dateparsed
Total Sources after DateTEST: bingnews                            72
bing                                50
Reuters                             38
Federal Reserve (Press Releases)    1 
Name: source, dtype: int64
4A.Size of Data BEFORE DATE FILTER applied: (161, 11)
DATE FILETER: TEST IF FAIL AFTER ROUGH DATE
4B.Size of Data After DATE FILTER applied: (23, 11)


In [60]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
df_step3 = newfunc(df_step2,comptoks=['Mizuho Bank','Mizuho','mizuho'])

5.Size of Data After ENTITY FOUND FILTER applied: (8, 16)
[]
6.Size of Data After Dup URL FILTER applied: (8, 16)


In [36]:
from nn_base.nn_translate import langname_to_iso639
from nn_base.nn_textprocess import entity_match_score, entity_stem_match,OrigTxt_PreProcess_DTCC
se = returnse(df_step3)
#list(df_step3)
#df_step3[['title','sentence_tokenized','sentence_tokenized_stemmed','sentence_tokenized_tagged']].head()

Size Post Data PreProcessing:  (66, 16)
Deleting 3 duplicate entries for entity
Error
ORIG TEXT PROCESSING DTCC RESULTS: origtext:63, modeltext:63


In [37]:
def asymptoc(x):
    x = (.3*x**2) / ((.3*x**2)+2)
    return x

def simasymptoc(x):
    x = (.1*x**2) / ((.1*x**2)+20)
    return x

df_step5 = w2v_se2(se, wv, 1,ndoc=400, nsent=400, sent_max = 0.3, min_sim = .15, delim = '')

hi!
[[['file', 'photo', 'headquarters', 'deutsche', 'bank', 'pictured', 'frankfurt', 'germany', 'march', 'reuters', 'ralph', 'orlowski', 'file', 'photo', 'frankfurt', 'reuters', 'germany', 'holding', 'high', 'level', 'talks', 'facilitate', 'possible', 'merger', 'nation', 'two', 'largest', 'private', 'banks', 'deutsche', 'bank', 'dbkgn', 'de', 'commerzbank', 'cbkg', 'de', 'bloomberg', 'reported', 'wednesday']], [['german', 'finance', 'minister', 'olaf', 'scholz', 'attends', 'press', 'conference', 'federal', 'cabinet', 'meeting', 'potsdam', 'germany', 'november', 'reuters', 'axel', 'schmidt', 'files', 'berlin', 'reuters', 'german', 'finance', 'minister', 'olaf', 'scholz', 'friday', 'lauded', 'positive', 'developments', 'german', 'banking', 'sector', 'recent', 'months', 'said', 'ministry', 'overly', 'concerned', 'future', 'deutsche', 'bank', 'germany', 'largest', 'bank'], ['talk', 'possible', 'merger', 'deutsche', 'bank', 'commerzbank', 'country'], ['bank', 'heated', 'past', 'weeks', 'len

In [ ]:
#pd.set_option('display.width', 10000)
#pd.set_option('display.max_columns', 500)
#scores = [(df_step5.entity_fname+"_"+df_step5.entity_lname,x[0][0],x[0][1],x[0][2],x[0][3],x[0][4],x[0][5],x[0][6],x[0][7],x[0][8],x[0][9],x[0][10],x[0][11]) for x in df_step5.w2vInfo[0]]
#flat_list = [sublist  for sublist in scores]#convert scores to dataframe and sort output by Doc -> Similarity 
#se_df = pd.DataFrame(flat_list, columns = ['Person','Doc','Sent','URL','Sentence','Similarity','Sentiment','textsummary','date','source','Title','EntityCounts','EntityFines'])
#se_df= se_df.sort_values(by=['Doc','Similarity'],ascending = False)
#se_df.to_csv("/home/nlpsomnath/NegNews/zackc/DTCC_Data/w2vInfo_BAC.csv")    
#se_df

In [38]:
#weight_func(x.DocSim,-1*x.DocSent,x.DocDateN,x.entityWeight,x.fineWeight), axis = 1)
listofranks = []

def weight_func(x,y,z,a,b):
    #print((x * 0.5) + (y * 0.5))
    return ((x * 0.85) + (y * .03) + (z * .00) + (a*.06) + (b*.06)) 
    
    #return ((x * 0.85) + (y * .03) + (z * .02) + (a*.05) + (b*.05)) 
    #return ((x * 0.93) + (y * .01) +(a*.02) + (b*.04)) 


df_step6 =  w2v_SortOutput(df_step5,1, max_sent = .3)

    Doc  Sent  entityWeight  EntityCounts
0   61   6     0.130435      1           
1   61   0     0.130435      1           
2   61   10    0.130435      1           
3   61   5     0.130435      1           
4   61   9     0.130435      1           
5   61   2     0.130435      1           
6   61   1     0.130435      1           
7   60   1     0.375000      2           
8   60   0     0.375000      2           
9   59   1     0.375000      2           
10  59   0     0.375000      2           
11  58   1     0.130435      1           
12  57   1     0.375000      2           
13  57   0     0.375000      2           
14  57   2     0.375000      2           
15  56   3     0.574468      3           
16  56   2     0.574468      3           
17  56   0     0.574468      3           
18  56   1     0.574468      3           
19  55   1     0.574468      3           
20  54   0     0.130435      1           
21  54   1     0.130435      1           
22  53   0     0.375000      2    

In [39]:
import numpy
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

#list(df_step6.w2vDfSorted[0])
#df_step6.w2vDfSorted[0][['Doc','NewsSource','Title','URL','date','DocSent','DocSim','EntityCounts','FinalDocScore','Final_Normalized_Score']].drop_duplicates()
#df_step6.w2vDfSorted[0][['Doc','NewsSource','Title']].drop_duplicates()
#df_step6.w2vDfSorted[0].to_csv("/home/nlpsomnath/NegNews/zackc/DTCC_Data/w2vInfoSorted_Mizo.csv") 
#df_step65=datesort(df_step6)
#df_step65=df_step65.sort_values(['dateint'], ascending=[False]).groupby(['Doc'],as_index=False)


In [40]:
#df_step6.w2vDfSorted[0].head(10)
#df_step6.w2vDfSorted[0][['Doc','Sent','FinalDocScore','maxscore','Final_Normalized_Score','maxscoreUN']]
df_step7 =rerank_searchresults_dt(df_step6,LSA_filter=0)
#df_step7 =rerank_searchresults(df_step6,LSA_filter=0)


THIS IS THE LENGTH OF W2vDoc 48
URL LIST BEFORE FILTER:  62
('https://in.reuters.com/article/germany-banks-scholz-idINKBN1OD27P', 'https://www.reuters.com/article/us-hna-group-assets-exclusive-idUSKBN1OD0V0', 'https://www.themarketsdaily.com/2018/12/14/deutsche-bank-ag-sells-129517-shares-of-hershey-co-hsy.html', 'https://www.themarketsdaily.com/2018/12/14/deutsche-bank-ag-has-79-64-million-position-in-republic-services-inc-rsg.html', 'https://in.reuters.com/article/deutsche-bank-moneylaundering-idINKBN1OC24I', 'https://www.law360.com/articles/1111048/deutsche-bank-not-liable-for-iran-bomb-7th-circ-says', 'https://finance.yahoo.com/news/daily-dicta-covington-wins-big-061850085.html', 'https://www.law360.com/articles/1111183/senate-dems-call-for-deutsche-bank-probe-after-aml-raid', 'https://uk.reuters.com/article/us-liberbank-m-a-unicaja-idUKKBN1OB171', 'https://www.businessinsider.com/deutsche-bank-shares-commerzbanks-merger-plans-2018-12', 'https://in.reuters.com/article/india-rbi-gov

In [41]:
df_step8 = Doccats_SummaryScores(df_step7)

In [42]:
#from nn_postprocessing import create_pdf_wrapper,empty_df_pdf
create_pdf_wrapper(df_step8)

/usr/local/lib/python3.5/dist-packages/docx/styles/styles.py:143: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


NewsSource:  ('in.reuters', 'reuters', 'themarketsdaily', 'themarketsdaily', 'in.reuters', 'law360', 'finance.yahoo', 'law360', 'uk.reuters', 'businessinsider', 'in.reuters', 'wvxu', 'thestreet', 'worldfinance', 'thestreet', 'fairobserver', 'news.yahoo', 'msn', 'heraldextra', 'in.reuters', 'in.reuters', 'jamaicaobserver', 'hartfordbusiness', 'lawprofessors.typepad', 'msn', 'news.bitcoin', 'thinkprogress', 'nypost', 'globalnews.ca', 'timesofindia.indiatimes', 'hurriyetdailynews', 'news.sky', 'therealdeal', 'thejakartapost', 'fastcompany', 'upi', 'wistv', 'nypost', 'news.efinancialcareers', 'newsok', 'euobserver', 'finance.yahoo', 'cityam', 'de.reuters', 'blogs.wsj', 'ww2.cfo', '247wallst', 'gulfnews')
PDF file stored at - 0_Deutsche Bank  2018-12-14-21-31-29_0.pdf
exception data not saved for given file path {} /home/nlpsomnath/NegNews/rebase/NN_Kafka/output_pdf/0_Deutsche Bank  2018-12-14-21-31-29_0.json
Data Not Saved


'/home/nlpsomnath/NegNews/rebase/NN_Kafka/output_pdf/0_DeutscheBank2018-12-14-21-31-29_0.pdf_2018-12-14-21-31-48_77.86'

### Process Wrappers 

In [6]:
#step1 
def df_preprocess(input_df,wv,IndvEnt_flag):
    wv = wv
    stop_words = set(stopwords.words('english'))


    df_rlt = input_df
    #TITLE PROCESSING
    #note:get_title() used for missing titles by using url response object
    df_rlt.loc[df_rlt['title'] == None, 'title'] = 'No Title Available'
    df_rlt.loc[df_rlt['title'] == np.nan, 'title'] = 'No Title Available'
    df_rlt.loc[df_rlt['title'].str.strip() == '', 'title'] = 'No Title Available'
    print('1.Title parsed',df_rlt.shape)
    
    #if config.use_dateparse==1:
    #    df_rlt['title']=df_rlt.apply(lambda row: get_title(url=row['url'])  if row['title']=="No Title Available" else row['title'], axis=1) 
  
    df_rlt["StripURL"] = df_rlt['url'].copy()
    replace_values = {'StripURL':{r'www.':'',r'https://':'',r'http://':'',r'.com':'',r'.org':'',r'.html':'',r'.gov':''}}
    df_rlt.replace(replace_values, regex=True, inplace=True)
    df_rlt["StripURL"]=df_rlt["StripURL"].map(lambda x: x[x.rfind('/')+1:] if len(x[x.rfind('/')+1:])>3 else x)
    df_rlt.loc[df_rlt['title'] == "No Title Available", 'title'] = df_rlt.loc[df_rlt['title'] == "No Title Available", 'StripURL']
    
    #convert elements from source data that are list into strings 
    #if any observation of title is found to be list, convert to string 
    df_rlt['title'] =df_rlt['title'].apply(lambda x: ' '.join(map(str, x)).strip() if isinstance(x,list) else x)
    df_rlt['date'] =df_rlt['date'].apply(lambda x: ' '.join(map(str, x)).strip() if isinstance(x,list) else x)

    #CONTENT AND SUMMARY PROCESSING
    df_rlt['summary'] =df_rlt['Summarylist'].apply(lambda x: ''.join(map(str, x)).strip())
    df_rlt = df_rlt.copy().drop(['Summarylist'], axis=1) 

    #if content is null copy summary over as proxy(NewsAPI does not return full content)
    df_rlt.loc[df_rlt['summary'].str.strip() == 'None', 'summary'] = df_rlt.loc[df_rlt['summary'].str.strip() == 'None', 'content']
    df_rlt.loc[df_rlt['summary'].str.strip() == '', 'summary'] = df_rlt.loc[df_rlt['summary'].str.strip() == '', 'content']
    df_rlt.loc[df_rlt['content'].str.strip() == '', 'content'] = df_rlt.loc[df_rlt['content'].str.strip() == '', 'summary']
    df_rlt=df_rlt[df_rlt.summary.str.strip() != ''].copy()
    df_rlt=df_rlt.copy().drop_duplicates()
    print('2.Size of Data After Empty Content/Summary Removed:',df_rlt.shape)
    
    #DATA PRESENCE CHECKPOINT 2
    if df_rlt.shape[0]<=1:
        raise Exception('SE object creation failed at data presence checkpoint 2')

    #if origtext = Nan use content 
    df_rlt ['origtext'].replace(np.nan,'', inplace=True)
    df_rlt.loc[df_rlt['origtext'] == '', 'origtext'] = df_rlt.loc[df_rlt['origtext']=='', 'content']
    df_rlt.loc[df_rlt['origtext'].str.strip() == '', 'origtext'] = 'No text'
    df_rlt.loc[df_rlt['origtext'] == 'No text', 'origtext'] = df_rlt.loc[df_rlt['origtext']=='No text', 'summary']
    df_rlt_complete = df_rlt[(df_rlt['url'].isnull()==False)].copy()
    
    print('Total Sources at end of URL based filtering and Content/Summary Filter:',df_rlt_complete.source.value_counts()) 
    return df_rlt_complete


#step 2
def enrich_date(df_rlt_complete,minyear,minmonth):
    #DATA PRESENCE CHECKPOINT 3
    if df_rlt_complete.shape[0]<=1:
        raise Exception('SE object creation failed at data presence checkpoint 3')
        
    #apply full date extraction function
    print('Total Sources at time of DateTEST:',df_rlt_complete.source.value_counts()) 
    #df_rlt_complete ['date'].replace('', np.nan, inplace=True)
   
    if config.use_dateparse ==1:
        df_rlt_complete['date_test'] = df_rlt_complete.apply(lambda row:  date_extraction(row['url'],row['origtext']) if pd.notna(row['date'])==False else row['date'], axis=1).astype(str)
        print('dateparsed')
    else: 
        df_rlt_complete['date_test'] =df_rlt_complete['date'].copy().astype(str)
    
    #extract date in consistent format , if date cannot be validated return nan 
    #print(df_rlt_complete['date_test'].unique())
    df_rlt_complete['date_test']=df_rlt_complete.apply(lambda x:  dateparser.parse(str(x["date_test"])).strftime("%m-%d-%Y") if validate_date(str(x["date_test"]))==True else np.nan , axis=1) # & pd.notna(x["date_test"])==True
    df_rlt_complete['date_test'] = pd.to_datetime(df_rlt_complete['date_test'], errors='coerce',format="%m-%d-%Y")
    print('Total Sources after DateTEST:',df_rlt_complete.source.value_counts()) 
    
    #Try to extract year from date  
    #reformat date from year to be inline with date column  
    df_rlt_complete['year'] = df_rlt_complete.apply(lambda x: rough_date_pandas_wrapper(x) if x['date_test']!= np.datetime64('NaT') else x['date_test'].year() , axis = 1)
    df_rlt_complete['year']= df_rlt_complete['year'].apply(lambda x: time.strftime("%m-%d-%Y", time.gmtime(time.mktime((int(str(x)[:4]), 1, 1, 17, 3, 38, 1, 48, 0)))) if (1970<int(str(x)[:4])<2050) else  time.strftime("%m-%d-%Y", time.gmtime(time.mktime((1899, 1, 1, 17, 3, 38, 1, 48, 0)))))
    df_rlt_complete['year'] = pd.to_datetime(df_rlt_complete['year'], format="%m-%d-%Y")

    #create consolidated date column 
    df_rlt_complete['date2']=df_rlt_complete.copy().apply(lambda x:  x["date_test"] if pd.notnull(x["date_test"]) else x['year'], axis=1)
    
    #drop intermediate date columns
    df_rlt_complete = df_rlt_complete.drop(['date','date_test'], axis=1)
    print('4A.Size of Data BEFORE DATE FILTER applied:',df_rlt_complete.shape)

    print('DATE FILETER: TEST IF FAIL AFTER ROUGH DATE')
    if config.minyear !='':
        df_rlt_complete['date_filter']=df_rlt_complete.apply(lambda x: x['date2'] >= datetime.datetime(minyear,minmonth,1),axis=1).copy()
        df_rlt_complete=df_rlt_complete[df_rlt_complete['date_filter']==True].copy()
        df_rlt_complete = df_rlt_complete.drop(['date_filter'], axis=1)  
    print('4B.Size of Data After DATE FILTER applied:',df_rlt_complete.shape)

    #DATA PRESENCE CHECKPOINT 4
    if df_rlt_complete.shape[0]<=1:
        raise Exception('SE object creation failed at data presence checkpoint 4')
    df_rlt_complete.to_pickle('//home//nlpsomnath//NegNews//zackc//DTCC_Data//fine_extract_test.pkl')
    
    return(df_rlt_complete)

#step3
def newfunc(df,comptoks=[]): 
    df_rlt_complete=df
    #force near filter(only pull back articles with entity name)
    #NEW FUNCITONALITY 
    IndvEnt_flag=1
    
    df_rlt_complete['sentence_tokenized'] = df_rlt_complete.apply(lambda x:sent_tokenize( transform_article_title(x['summary']+' '+x['title'],company_tokens=comptoks)), axis=1)
    #df_rlt_complete['sentence_tokenized_stemmed'] = df_rlt_complete['sentence_tokenized'].apply(lambda x: [' '.join([stemmer.stem(j) for j in word_tokenize(x[i])]) for i in range(0,len(x))])
    df_rlt_complete['sentence_tokenized_tagged'] = df_rlt_complete.apply(lambda x: list(map(lambda sent: nlp(sent), x['sentence_tokenized'])), axis=1)

    df_rlt_complete['entity_count'] = df_rlt_complete.apply(lambda x: sum(list(map(lambda sent: max([fuzz.partial_token_set_ratio(sent, name) > ENTITY_FUZZ_SCORE_THRESHOLD for name in known_alias]), x['sentence_tokenized']))), axis=1)
    df_rlt_complete['fine_info_tuple_lists'] = df_rlt_complete.apply(lambda x: list(map(lambda sent: get_data_from_article_title(entity_dict, tagged_text=sent, entity_name=df_rlt_complete.entity.unique()[0]), x['sentence_tokenized_tagged'])), axis=1)
    df_rlt_complete['total_entity_fine_amt'] = df_rlt_complete.apply(lambda x: sum(set([t[FINE_AMOUNT_IDX] if t[FINE_GENERATE_FLAG_IDX] else 0 for t in x['fine_info_tuple_lists']])), axis=1)
  
                                                                                 
    
    #will original 
    #df_rlt_complete['sentence_tokenized_tagged'] = df_rlt_complete.apply(lambda x: list(map(lambda sent: nlp(sent), x['sentence_tokenized'])), axis=1)
    #df_rlt_complete['fine_info_tuple_lists'] = df_rlt_complete.apply(lambda x: list(map(lambda sent: get_data_from_article_title(entity_dict, tagged_text=sent, entity_name=df_rlt_complete.entity.unique()[0]), x['sentence_tokenized_tagged'])), axis=1)
    #df_rlt_complete['total_entity_fine_amt'] = df_rlt_complete.apply(lambda x: sum(set([t[FINE_AMOUNT_IDX] if t[FINE_GENERATE_FLAG_IDX] else 0 for t in x['fine_info_tuple_lists']])), axis=1)
    df_rlt_complete.to_pickle('//home//nlpsomnath//NegNews//zackc//DTCC_Data//DB_fine_extract_test.pkl')
    
    #print(df_rlt_complete.loc[:,['fine_info_tuple_lists','total_entity_fine_amt']])
    #df_rlt_complete=df_rlt_complete[df_rlt_complete['EntityFound']==True].copy()
    df_rlt_complete=df_rlt_complete[df_rlt_complete['entity_count']>0].copy()
    print('5.Size of Data After ENTITY FOUND FILTER applied:',df_rlt_complete.shape)

    #drop duplicate urls 
    dup_idxs=url_fuzzy_dupes(df_rlt_complete['url'])
    print(dup_idxs)
    df_rlt_complete.drop(df_rlt_complete.index[dup_idxs],inplace=True)
    #df_rlt_complete.drop_duplicates(subset=['url'], keep='first', inplace=True)
    #df_rlt_complete = df_rlt_complete.drop(['EntityFound'], axis=1)    
    print('6.Size of Data After Dup URL FILTER applied:',df_rlt_complete.shape)

    #DATA PRESENCE CHECKPOINT 5
    if df_rlt_complete.shape[0]<=1:
        raise Exception('SE object creation failed at data presence checkpoint 5')     

    return(df_rlt_complete)

#step4
def returnse(df):
    df_rlt_complete = df
    #save Pandas to json
    #df_rlt_complete.to_json(config.mongostage_dir+time.strftime("%Y-%m-%d-%H-%M-%S")+'_'+'.JSON')
    data = df_rlt_complete.copy()
    print('Size Post Data PreProcessing: ',data.shape)

    #### BEGIN MUELLER EDITS 8/30 ####
    dupe_idx = MinHash(data['content']).idx_to_remove
    data.drop(data.index[dupe_idx], inplace = True)
    print('Deleting {} duplicate entries for entity'.format(len(dupe_idx)))
    #### END MUELLER EDITS 8/30 ####

    
    #inialize SE OBJECT 
    name= df_rlt_complete.entity.unique()[0]
    fname,lname = namesplit(name,entity=config.entity)   

    #initalize se with required parameters
    se = SE(searchid = str(0)
            , search_lang = ['en']
            , fname = fname
            , lname=lname
            , searchtermlist=[config.searchterms1]
        )

    #add optional parameters to se object
    se.search_lang_short = list(map(langname_to_iso639, se.search_lang))
    textlistTemp = se.origtextlist[0] = list(data['origtext'])
    urllist = se.urllist[0] = list(data['url']) #list of comma seperated URLs
    datelist = se.lastsearchdate[0] = list(data['date2'])
    summarylist = se.textsummary[0] = list(data['summary'])
    entitylist = se.NewsSource[0] = list(data['source'])
    querylist = ','.join(list(set([x for x in list(data['source'])])))
    se.entity_querylist = se.querylisturl[0] = entity_querylist = querylist
    titlelist = se.DocTitle[0] = list(data['title'])
    
    #new data for DTCC 
    se.entitycount[0] = list(data['entity_count'])
    se.entityfines[0] = list(data['total_entity_fine_amt'])
    
    searchterm_list  = [config.searchterms1] #if config.termlist==1 else [config.searchterms2] #translate_searchterms(config.searchterms2) if config.termlist==2 else translate_searchterms(config.searchterms1)
    #print('Post generation of list attributes:','\n',len(list(filter(None,textlistTemp))),len(textlistTemp),'\n',len(list(filter(None,urllist))),len(urllist),'\n')
   
    try: 
        se.entity_company= [x.strip() for x in df_rlt_complete.jobname.unique()[0].split('_')][2]
        se.entity_state= [x.strip() for x in df_rlt_complete.jobname.unique()[0].split('_')][1]
    except Exception: 
        print('Error')
    
    #print('Post generation of se list attributes:',"\n",len(list(filter(None,se.origtextlist[0]))),len(list(filter(None,se.urllist[0]))))

    #sent tokenize and preprocess text for modeling
    textlist,origtext=OrigTxt_PreProcess_DTCC(textlistTemp, se.entity_fname,se.entity_lname,delim ='', window=False)
    textlist_zip = list(zip(textlist, origtext, se.urllist[0],se.textsummary[0],se.lastsearchdate[0]
                            ,se.NewsSource[0],se.DocTitle[0], se.entityfines[0],se.entitycount[0]))

    #remove empty values from each list , assign each element of tuple to individual tuples of all values of the tuple element in question and convert each to list and assign to se
    info_tuples = [(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]) for x in textlist_zip if len(x[0]) != 0] 
    t1,t2,t3,t4,t5,t6,t7,t8,t9= zip(*info_tuples)
    se.textlist[0]=list(t1)
    se.pdftext[0]=list(t2)
    se.urllist[0]=list(t3)
    se.textsummary[0] = list(t4)
    se.lastsearchdate[0] = list(t5)
    se.NewsSource[0] = list(t6)
    se.DocTitle[0] = list(t7)
    se.entityfines[0] = list(t8)
    se.entitycount[0] = list(t9)
    
    
    #print('Post filtering length check: ',len(list(filter(None,se.textlist[0]))),len(se.textlist[0]),len(list(filter(None,se.pdftext[0]))),len(se.pdftext[0]),len(list(filter(None,se.urllist[0]))))

    #4) Calculate relevancy scores(THESE NEED TO BE UPDATED)
    scoreslist = [entity_match_score(text, se, 0) for text in se.textlist[0]]

    #5) Find stem matches
    stemmatchlist = [entity_stem_match(' '.join(text).split(), se, 0) for text in se.textlist[0]]

    #6) Save the processed results (cleaned text, fuzzyscores, details, and matched stems) into appropriate se slot
    se.list_fuzzyscore[0] = [np.mean([x for x in scores if x > 0]) if len([x for x in scores if x > 0])>0 else 0 for scores in scoreslist]
    se.list_fuzzyscoredetail[0] = scoreslist
    se.list_matchedstems[0] = stemmatchlist
    #print('urls:',len(se.urllist[0]),'orig:',len(se.textlist[0]),'pdf:',len(se.pdftext[0]))
    #print('Origurls:',len(urllist),'orig:',len(se.textlist[0]),'pdf:',len(se.pdftext[0]))

    for idx, text in enumerate(se.textlist):
                if text == []: se.urllist[idx]=[]
    #print('URL length after check for text presence:',len(se.urllist[0]))

    searchresults = ' '.join(['{}:{}'.format(x,len(y)) for (x,y) in zip(se.search_lang, se.urllist)])
    #print('Searched {} languages total. The results are: {}'.format(se.search_lang_ct, searchresults)) 
    #print('TEXT BEFORE INPUT TO W2V ' , se.textlist[0])
    
    return se 

### W2V and Ranking Functions

In [24]:

def w2v_se2(se, wv, entity,ndoc=400, nsent=400, sent_max = 0.3, min_sim = .1, delim = ''):
    #Not#e: we compile the function here and execute the function at the bottom of the parent function 

    def w2v_SimSent(seurllist, doc_cluster, stems, wv,ndoc,nsent, sent_max, min_sim, se,delim=''):
            # print(len(doc_cluster))
            # print(len(seurllist))
            #text procesed by w2v vs original sentence to be passed to vadar
            text_list = clean_paragraph(doc_cluster,'english',stemming=False,sent_tokenize=True, rem_stop=True , delim = delim)
            print('hi!')
            print(text_list)
            text_list_vader = clean_paragraph(doc_cluster,'english',stemming=False,sent_tokenize=True, rem_stop=False, delim =delim) #note: both text list, 
            text_summary = se.textsummary[0]
            date = se.lastsearchdate[0]
            source = se.NewsSource[0]
            title = se.DocTitle[0]
            entitycount = se.entitycount[0]
            entityfines = se.entityfines[0]
            
            def simasymptoc(x):
                x = (.6 +x**2) / ((3+x**2))
                return x
            
            print('DocCluster_TxtlistCounts(All,NonNull): ',len(doc_cluster),len(text_list)
            ,len(list(filter(None,doc_cluster))),len(list(filter(None,text_list))))

            #tfidf 
    #         t = tf_idf(text_list, stems)

            #Get average word vector for search terms
    #         search_vec = avg_phrase(stems, wv, t["tf_idf"], t["dic"], 0)
            z = stem_phrase(stems,wv)

            #W2V Execution 
            #compute average word embeddings and identify cosine similarity of the average word vector of the sentence vs average vector of search tersm     
            from datetime import datetime
            startTime = datetime.now()
            sim_list2 = []

            for idx, val in enumerate(text_list):
                #print first text ;if idx==1:print(val)
                if len(val) > 0 and idx < ndoc:

                    #for each sentence in each document 
                    for idx2, sent in enumerate(val):
                        sim = 0
                        sim_num = 0
                        #get sent score up front for downstream use 
                        Vad_Score = VADERSENT(' '.join(text_list_vader[idx][idx2]),many=False)[0][1]

    #                     #generate the avg vector associated with each sent in doc idx+1 (because idx(0) is the stems)
    #                     sent_vec = avg_phrase(sent, wv, t["tf_idf"],t["dic"],idx+1)

    #                     #could also try including VADER filter here (got error last time)
    #                     if sent_vec.all != 0 and idx2 < nsent: 

    #                         #error handling to prevent computation of cosine similarity with negative numbers 
    #                         try:                           
    #                             sim = cosine_sim(search_vec, sent_vec)
                        sim, sim_num = avg_phrase(sent, wv, z)
    #                     print(sim, sim_num, sim_final)
                        sim_final = sim*simasymptoc(sim_num)
                        print(sim, sim_num, sim_final)
                        #create one metric for each sentence with asymptopic function from 0 to 1.



                        sent_sim2_all = [sim_final] #Method 0: one metric for each sentrence (averaged across all stems) #NORMALIZING BIAS SHORT STRINGS FOR LONG ONES REGADLESS   
    #                         except: 
    #                             sent_sim2_all = [0] #REMOVE THIS, THIS IS NOT GOOD 

                            #only extract sentences that meet similarity and sentiment thresholds
                        if sim > min_sim and Vad_Score < sent_max :#and config.indiv_parse==0
                            sim_list2.append(list(zip([idx],[idx2],[seurllist[idx]],[' '.join(text_list[idx][idx2])],sent_sim2_all, [Vad_Score],[text_summary[idx]],[date[idx]],[source[idx]],[title[idx]],[entitycount[idx]],[entityfines[idx]]))) #Without URL Included 

            #total #time required to run algo 
            print(datetime.now() - startTime)
            return(sim_list2)


    #for each languge if the ersults are not blank, extract the post processed text, and clean the search terms , and join them into one string
    for idxx in range(0,se.search_lang_ct):
        if not se.urllist[idxx] == []:
            #pull out original text from se object 
            doc_cluster = se.textlist[idxx]
            stems= clean_paragraph(se.searchtermlist[idxx], se.search_lang_short[idxx], stemming=False, sent_tokenize=False, rem_stop=True)
            #execute the function for each languge 
            se.w2vInfo[idxx]  = w2v_SimSent(se.urllist[idxx],  doc_cluster, stems, wv, ndoc=ndoc, nsent=nsent, sent_max=sent_max, min_sim = min_sim, se=se)    

    return se


##NEW 12/8/18 DOC LEVEL and PERSON LEVEL
def w2v_SortOutput(se, entity, max_sent = config.max_sent): 
    def W2V_ScoreAgg(fname, lname, se_w2vInfo, entity,max_sent):
    
        scores = [(se.entity_fname+"_"+se.entity_lname,x[0][0],x[0][1],x[0][2],x[0][3],x[0][4],x[0][5],x[0][6],x[0][7],x[0][8],x[0][9],x[0][10],x[0][11]) for x in se.w2vInfo[0]]
        flat_list = [sublist  for sublist in scores]#convert scores to dataframe and sort output by Doc -> Similarity 
        se_df = pd.DataFrame(flat_list, columns = ['Person','Doc','Sent','URL','Sentence','Similarity','Sentiment','textsummary','date','source','Title','EntityCounts','EntityFines'])
        se_df= se_df.sort_values(by=['Doc','Similarity'],ascending = False)
        
        
        #Sort se_df by Document and take the three lowest sentiment sentences values and average
        doc_levelSE = se_df.sort_values(['Doc','Sentiment'], ascending=[True,True]).groupby(['Doc'],as_index=False).head(3)
        doc_levelSE = doc_levelSE.groupby(by=['Doc'],as_index=False)['Sentiment'].mean() ; doc_levelSE.fillna(0, inplace=True)
        
        #Sort se_df by Document and take the three highest similarity sentence values and average
        doc_levelSI = se_df.sort_values(['Doc','Similarity'], ascending=[True,False]).groupby(['Doc'],as_index=False).head(3)
        doc_levelSI = doc_levelSI.groupby(by=['Doc'],as_index=False)['Similarity'].mean() ; doc_levelSI.fillna(0, inplace=True)
        
        #merge the above sentiment and similarity back with se_df (sure this can be done more cleanly)
        doc_level1 = se_df
        
        doc_levelSE['DocSent'] = doc_levelSE.Sentiment
        doc_levelSI['DocSim'] = doc_levelSI.Similarity
        doc_level1 = doc_level1.merge(doc_levelSE[['Doc','DocSent']], how='left', left_on='Doc', right_on='Doc')
        doc_level1 = doc_level1.merge(doc_levelSI[['Doc','DocSim']], how='left', left_on='Doc', right_on='Doc')
        
        #compute daterank by taking time delta to get absolute time number
        doc_level1['daterank']= pd.to_timedelta(pd.to_datetime(doc_level1.date)).dt.total_seconds()#.astype(int)
        
        #normalise the time range (only normalise date at this point as it is a huge number otherwise)
        doc_level1['DocDateN'] = ((doc_level1.daterank - doc_level1.daterank.min()) / (doc_level1.daterank.max()-doc_level1.daterank.min())) #normalized doc score
        
        ##ADD in the Entity and Fine Ranking/weighting here
        doc_level1['entityWeight'] = asymptoc(doc_level1.EntityCounts) #check this field is correct 
        doc_level1['fineWeight'] = ((doc_level1.EntityFines - doc_level1.EntityFines.min()) / (doc_level1.EntityFines.max()-doc_level1.EntityFines.min())) #normalized doc score
        doc_level1['fineWeight'].fillna(0,inplace=True)
    
        #Binning for Sent, Sim and Date
        doc_level1.loc[doc_level1.DocSim < 0.14, 'DocSimStr'] = 'Low'
        doc_level1.loc[(doc_level1.DocSim >= 0.14) & (doc_level1.DocSim < 0.172), 'DocSimStr'] = 'Med'
        doc_level1.loc[doc_level1.DocSim >= 0.172, 'DocSimStr'] = 'High'
        
        doc_level1.loc[doc_level1.DocSent < -0.4, 'DocSentStr'] = 'High'
        doc_level1.loc[(doc_level1.DocSent >= -0.4) & (doc_level1.DocSent < -0.18), 'DocSentStr'] = 'Med'
        doc_level1.loc[doc_level1.DocSent >= -0.18, 'DocSentStr'] = 'Low'
        
        doc_level1.loc[doc_level1.DocDateN < 0.37, 'DocDateStr'] = 'Low'
        doc_level1.loc[(doc_level1.DocDateN >= 0.37) & (doc_level1.DocDateN < 0.7), 'DocDateStr'] = 'Med'
        doc_level1.loc[doc_level1.DocDateN >= 0.7, 'DocDateStr'] = 'High'
        
        doc_level1.loc[doc_level1.entityWeight < 0.3, 'DocCountStr'] = 'Low'
        doc_level1.loc[(doc_level1.entityWeight >= 0.3) & (doc_level1.entityWeight < 0.4), 'DocCountStr'] = 'Med'
        doc_level1.loc[doc_level1.entityWeight >= 0.4, 'DocCountStr'] = 'High'
        
        doc_level1.loc[doc_level1.fineWeight <= 0, 'DocFineStr'] = 'Low'
        doc_level1.loc[(doc_level1.fineWeight > 0) & (doc_level1.fineWeight < 0.4), 'DocFineStr'] = 'Med'
        doc_level1.loc[doc_level1.fineWeight >= 0.4, 'DocFineStr'] = 'High'
        
        print(doc_level1[doc_level1['entityWeight']>0].loc[:,['Doc','Sent','entityWeight','EntityCounts']])#['fineWeight','EntityFines']])
        #save data
        doc_level1.to_pickle('//home//nlpsomnath//NegNews//zackc//DTCC_Data//datab4totalscore.pkl')
    
        doc_level1['count_max'] = asymptoc(doc_level1.groupby(['Person','Doc','URL'],as_index=False)['Sent'].transform(max))
        
        #apply the weighting function to similarity and sentiment (to stems) and date. ######ADD # OF TIMES ENTITY FOUND HERE
        doc_level1['maxscoreUN'] = doc_level1.apply(lambda x: weight_func(x.DocSim,-1*x.DocSent,x.DocDateN,x.entityWeight,x.fineWeight), axis = 1)
        
        #normalise the maxscores but keep both raw and normalized for later
        doc_level1['maxscore'] = (doc_level1.maxscoreUN - doc_level1.maxscoreUN.min()) / (doc_level1.maxscoreUN.max()-doc_level1.maxscoreUN.min()) #normalized doc score
        
        #print(doc_level1)
        #computing sentence count correctly, dont need this as of now.
        doc_test = se_df
        
        #find max of all the values (I think this can be removed  now) ####ADD in entity and fine rank here
        doc_level1 = doc_level1.groupby(by=['Person','Doc','URL']
                                        ,as_index=True)['maxscore','maxscoreUN','DocSim','DocSimStr'
                                                        ,'DocSent','DocSentStr','DocDateN','DocDateStr' 
                                                        ,'EntityCounts','DocCountStr'
                                                        ,'EntityFines','DocFineStr'
                                                       ].apply(
            lambda x : x.max()).reset_index() ; doc_level1.fillna(0, inplace=True)
        
        #assign maxscore to finaldocscore and normalize for ranking purposes
        doc_level1['FinalDocScore'] =doc_level1.maxscore
    
        doc_level1['FinalDocScore'] = (doc_level1.FinalDocScore - doc_level1.FinalDocScore.min()) / (doc_level1.FinalDocScore.max()-doc_level1.FinalDocScore.min()) #normalized doc score
        
        
        # display(doc_level1)
        doc_level1 = doc_level1.sort_values(by=['FinalDocScore','maxscoreUN','Doc'],ascending = False)
        # doc_level1 = doc_level1.sort_values(by=['FinalDocScore','maxscoreUN','DocSim','DocSent','DocDate','Doc'],ascending = False)
        
        #merge normalised (finaldocscore) and unnormalized doc scores back to original df. Sort by FinalDocScore
        se_df_sort = se_df.merge(doc_level1[['FinalDocScore','maxscoreUN','DocSim','DocSimStr'
                                             ,'DocSent','DocSentStr','DocDateN','DocDateStr','Doc'
                                             ,'DocCountStr','DocFineStr','maxscore']]
                                 , how='left', left_on='Doc', right_on='Doc')
        
        se_df_sort = se_df_sort.sort_values(by=['FinalDocScore','Doc','Similarity'],ascending = False)
        
        #Original TotalLinks and TotalSents (we use the TotalLinks)
        Person_level = se_df_sort.groupby(by=['Person'],as_index=False)['Similarity','Sentiment'].sum()
        Person_level['TotalLinks'] = se_df_sort.groupby(by=['Person'],as_index=False).Doc.nunique();Person_level.fillna(0,inplace=True) #very distinct distributions(bad people have more hits)
        Person_level['TotalSents'] = se_df_sort.groupby(by=['Person'],as_index=False).Sent.nunique();Person_level.fillna(0,inplace=True) #this is wrong 
        
        #Grab the top 20 articles for final doc ranking to weed out any potential bad articles later on. 
        Person_level1 = doc_level1.sort_values(['maxscoreUN','Doc'], ascending=[False,True]).head(20)#takes only the top 20 documents
        
        #Take mean of the top 20 for Person avg max score. Multiple by asymtop func weighing of total links to get final person score (un-normalised)
        Person_level1 = Person_level1.groupby(by=['Person'],as_index=False)['maxscoreUN'].mean() ; Person_level1.fillna(0, inplace=True)
        Person_level['Avg_MaxScore'] = Person_level1.maxscoreUN*asymptoc(Person_level.TotalLinks)
        
        #New ranges based on testing as of 12/9/2018
        MinVal = 0.1
        MaxVal = 0.264    
        
        #generate normalized score using above Min/Max normal curve limits
        Person_level['Final_Normalized_Score'] = max(0,min(100,(Person_level['Avg_MaxScore'].unique()[0] -MinVal) / (MaxVal-MinVal)))
        
        #final df output: merges info from all levels of data(sent , doc, person) 
        se_df_sort2 = se_df_sort.merge(Person_level[['Final_Normalized_Score','Person']], how='left', left_on='Person', right_on='Person')
      
        #se_df_sort2 = se_df_sort2.sort_values(by=['Doc','Sent'], ascending = [True,False])
        se_df_sort2['RegNonReg'] = [RegVsUnreg(url)[0] for url in se_df_sort2['URL']]
        se_df_sort2['NewsSource'] = [RegVsUnreg(url)[1] for url in se_df_sort2['URL']]
        print('Size of data after w2v sorting:{}'.format(len(se_df_sort2.URL.unique())))
     
        
        return(se_df_sort2)
    
    #for each languge if the ersults are not blank, extract the post processed text, and clean the search terms , and join them into one string
    for idxx in range(0,se.search_lang_ct):
        if not se.urllist[idxx] == []: 
            #execute the function for each languge 
            se.w2vDfSorted[idxx]  = W2V_ScoreAgg(se.entity_fname, se.entity_lname, se.w2vInfo[idxx], entity, max_sent)
    return(se)

def to_integer(dt_time):
    return int(10000*dt_time.year + 100*dt_time.month + dt_time.day)+1

    
def datesort(se):
    for idx in range(0, se.search_lang_ct):
        if not se.urllist[idx] == []:

            #Extract items to output from w2v
            df_person = se.w2vDfSorted[idx]
            df_person['dateint']= df_person.apply(lambda x: to_integer(pd.to_datetime(x.date)),axis=1)

            df_person.sort_values('dateint',ascending=False).groupby(['Doc'])
    return df_person
            
#these results are only used in nn_pdf to genrate the pdf 
def rerank_searchresults(se, LSA_filter=-100):
    for idx in range(0, se.search_lang_ct):
        if not se.urllist[idx] == []:

            #Extract items to output from w2v
            df_person = se.w2vDfSorted[idx]
            
            #*assume origlist is the list of orgtext where index matches to doc/sent on w2vfSorted
            df_person['pdftext'] = df_person.apply(lambda x: se.pdftext[idx][x.Doc][x.Sent], axis=1)
            
            #Rank of documents
            w2vDocRank= list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['FinalDocScore'].max()) #maxscoreUN
            #Most important sentences in each document
            w2vTopSent= list(df_person.sort_values(['Doc','Similarity'], ascending=[True, False]).groupby(['Doc'])['pdftext'].apply(list).apply(lambda x: x[0:3]))
            #url of each doc - already have, check if sorting was done right
            w2vdocs = list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['Doc'].first())
            print('THIS IS THE LENGTH OF W2vDoc',len(w2vdocs))
            textsummary =list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['textsummary'].first())
            date =list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['date'].first())
            source =list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['source'].first()) #CNN , etc.
            print('date type:',[type(x) for x in date])

            #regvsnon
            regnoreg = list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['RegNonReg'].first())
            NewsSource = list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['NewsSource'].first()) #this is the actual url root 
            Title = list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['Title'].first())
            
            #new
            DocSentStr= list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['DocSentStr'].first())
            DocSimStr= list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['DocSimStr'].first())
            DocDateStr= list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['DocDateStr'].first())
            DocCountStr= list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['DocCountStr'].first())
            DocFineStr= list(df_person.sort_values(['Doc','FinalDocScore'], ascending=[True, False]).groupby(['Doc'])['DocFineStr'].first())

            w2vDocRank_full = []
            w2vTopSent_full = []
            textsummary_full = []
            RegNoReg = []
            News = []
            dates =[]
            sources=[]
            titles=[]
            docsentstr = []
            docsimstr = []
            docdatestr = []
            doccountstr = []
            docfinestr = []
            
            counter = 0
            print('URL LIST BEFORE FILTER: ',len(se.urllist[idx]))
            
            for idxx, value in enumerate(se.urllist[idx]):
                
                if idxx in w2vdocs:
                    w2vDocRank_full.append(w2vDocRank[counter])
                    w2vTopSent_full.append(w2vTopSent[counter])
                    RegNoReg.append(regnoreg[counter])
                    News.append(NewsSource[counter])
                    textsummary_full.append(textsummary[counter])
                    dates.append(date[counter])
                    sources.append(source[counter])
                    titles.append(Title[counter])
                    docsentstr.append(DocSentStr[counter])
                    docsimstr.append(DocSimStr[counter])
                    docdatestr.append(DocDateStr[counter])
                    doccountstr.append(DocCountStr[counter])
                    docfinestr.append(DocFineStr[counter])

                    counter+=1
                else:
                    w2vDocRank_full.append(-9999)
                    w2vTopSent_full.append('')
                    RegNoReg.append('')
                    News.append('')
                    textsummary_full.append('')
                    dates.append('')
                    sources.append('')
                    titles.append('')
                    docsentstr.append('')
                    docsimstr.append('')
                    docdatestr.append('')
                    doccountstr.append('')
                    docfinestr.append('')

            se.w2vDocRank[idx]=w2vDocRank_full
            se.w2vTopSent[idx]=w2vTopSent_full
            se.RegNonReg[idx]=RegNoReg
            se.NewsSource[idx]=News
            se.source[idx] = sources
            se.textsummary[idx]=textsummary_full
            se.lastsearchdate[idx]=dates
         
            se.DocTitle[idx]=titles
            se.DocSentStr[idx]=docsentstr
            se.DocSimStr[idx]=docsimstr
            se.DocDateStr[idx]=docdatestr
            se.DocCountStr[idx]=doccountstr
            se.DocFineStr[idx]=docfinestr
            
            
            
            
            whole_list = list(zip(se.w2vDocRank[idx], se.urllist[idx],se.w2vTopSent[idx]
                                  , se.list_fuzzyscore[idx],se.list_matchedstems[idx]
                                  , se.list_fuzzyscoredetail[idx], se.textlist[idx]
                                  , se.RegNonReg[idx],se.NewsSource[idx],se.textsummary[idx]
                                  , se.lastsearchdate[idx],se.source[idx], se.DocTitle[idx]
                                  , se.DocSentStr[idx],se.DocSimStr[idx],se.DocDateStr[idx]
                                  , se.DocCountStr[idx],se.DocFineStr[idx]
                                 ))
            sorted_list = sorted(whole_list, key = operator.itemgetter(0), reverse = True)
            transposed_sorted = list(zip(*sorted_list))
            #print(transposed_sorted)
            #can be used to rerank and score docs diff
            LSA_filter_count = len([x for x in transposed_sorted[0] if x >= LSA_filter])
            se.LSA_filter_count[idx] = LSA_filter_count
            ## Extract the links and scores
            if LSA_filter_count > 0:
                se.w2vDocRank[idx] = transposed_sorted[0][0:LSA_filter_count]
                se.urllist[idx] = transposed_sorted[1][0:LSA_filter_count]
                se.w2vTopSent[idx] = transposed_sorted[2][0:LSA_filter_count]
                se.list_fuzzyscore[idx] = transposed_sorted[3][0:LSA_filter_count]
                se.list_matchedstems[idx] = transposed_sorted[4][0:LSA_filter_count]
                se.list_fuzzyscoredetail[idx] = transposed_sorted[5][0:LSA_filter_count]
                se.textlist[idx] = transposed_sorted[6][0:LSA_filter_count]
                se.RegNonReg[idx] = transposed_sorted[7][0:LSA_filter_count]
                se.NewsSource[idx] = transposed_sorted[8][0:LSA_filter_count]
                se.textsummary[idx] = transposed_sorted[9][0:LSA_filter_count]
                se.lastsearchdate[idx] = transposed_sorted[10][0:LSA_filter_count]
                se.source[idx] = transposed_sorted[11][0:LSA_filter_count]
                se.DocTitle[idx] = transposed_sorted[12][0:LSA_filter_count]
                
                se.DocSentStr[idx]= transposed_sorted[13][0:LSA_filter_count]
                se.DocSimStr[idx]= transposed_sorted[14][0:LSA_filter_count]
                se.DocDateStr[idx]= transposed_sorted[15][0:LSA_filter_count]
                se.DocCountStr[idx]= transposed_sorted[16][0:LSA_filter_count]
                se.DocFineStr[idx]= transposed_sorted[17][0:LSA_filter_count]

            else:
                se.w2vDocRank[idx] = []
                se.urllist[idx] = []
                se.w2vTopSent[idx] = []
                se.list_fuzzyscore[idx] = []
                se.list_matchedstems[idx] = []
                se.list_fuzzyscoredetail[idx] = []
                se.textlist[idx] = []
                se.RegNonReg[idx] =[] 
                se.NewsSource[idx] =[]
                se.textsummary[idx] =[]
                se.lastsearchdate[idx] =[]
                se.source[idx] = []
                se.DocTitle[idx]=[]
                se.DocSentStr[idx]= []
                se.DocSimStr[idx]= []
                se.DocDateStr[idx]= []
                se.DocCountStr[idx]= []
                se.DocFineStr[idx]=[] 
                
            print(se.urllist[idx])
            print('=====')
            print(transposed_sorted[7])
            print('NEW URL LIST LENGTH:',len(se.urllist[idx]))
    return se




#these results are only used in nn_pdf to genrate the pdf 
def rerank_searchresults_dt(se, LSA_filter=-100):
    
    for idx in range(0, se.search_lang_ct):
        if not se.urllist[idx] == []:

            #Extract items to output from w2v
            df_person = se.w2vDfSorted[idx]
            df_person['dateint']= df_person.apply(lambda x: to_integer(pd.to_datetime(x.date)),axis=1)
            
            #*assume origlist is the list of orgtext where index matches to doc/sent on w2vfSorted
            df_person['pdftext'] = df_person.apply(lambda x: se.pdftext[idx][x.Doc][x.Sent], axis=1)
            df_person.sort_values('dateint',ascending=False,inplace=True)
            
            #Rank of documents
            w2vDocRank= list(df_person.sort_values(['dateint'], ascending=[ False]).groupby(['Doc'])['dateint'].max()) #maxscoreUN
            #Most important sentences in each document
            w2vTopSent= list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['pdftext'].apply(list).apply(lambda x: x[0:3]))
            #url of each doc - already have, check if sorting was done right
            w2vdocs = list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['Doc'].first())
            print('THIS IS THE LENGTH OF W2vDoc',len(w2vdocs))
            textsummary =list(df_person.sort_values(['dateint'], ascending=[ False]).groupby(['Doc'])['textsummary'].first())
            date =list(df_person.sort_values(['dateint'], ascending=[ False]).groupby(['Doc'])['date'].first())
            source =list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['source'].first()) #CNN , etc.

            #regvsnon
            regnoreg = list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['RegNonReg'].first())
            NewsSource = list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['NewsSource'].first()) #this is the actual url root 
            Title = list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['Title'].first())
            
            #new
            DocSentStr= list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['DocSentStr'].first())
            DocSimStr= list(df_person.sort_values(['dateint'], ascending=[False]).groupby(['Doc'])['DocSimStr'].first())
            DocDateStr= list(df_person.sort_values(['dateint'], ascending= [False]).groupby(['Doc'])['DocDateStr'].first())
            DocCountStr= list(df_person.sort_values(['dateint'], ascending= [False] ).groupby(['Doc'])['DocCountStr'].first())
            DocFineStr= list(df_person.sort_values(['dateint'], ascending= [False]).groupby(['Doc'])['DocFineStr'].first())

            w2vDocRank_full = []
            w2vTopSent_full = []
            textsummary_full = []
            RegNoReg = []
            News = []
            dates =[]
            sources=[]
            titles=[]
            docsentstr = []
            docsimstr = []
            docdatestr = []
            doccountstr = []
            docfinestr = []
            
            counter = 0
            print('URL LIST BEFORE FILTER: ',len(se.urllist[idx]))
            
            for idxx, value in enumerate(se.urllist[idx]):
                
                if idxx in w2vdocs:
                    w2vDocRank_full.append(w2vDocRank[counter])
                    w2vTopSent_full.append(w2vTopSent[counter])
                    RegNoReg.append(regnoreg[counter])
                    News.append(NewsSource[counter])
                    textsummary_full.append(textsummary[counter])
                    dates.append(date[counter])
                    sources.append(source[counter])
                    titles.append(Title[counter])
                    docsentstr.append(DocSentStr[counter])
                    docsimstr.append(DocSimStr[counter])
                    docdatestr.append(DocDateStr[counter])
                    doccountstr.append(DocCountStr[counter])
                    docfinestr.append(DocFineStr[counter])

                    counter+=1
                else:
                    w2vDocRank_full.append(-9999)
                    w2vTopSent_full.append('')
                    RegNoReg.append('')
                    News.append('')
                    textsummary_full.append('')
                    dates.append('')
                    sources.append('')
                    titles.append('')
                    docsentstr.append('')
                    docsimstr.append('')
                    docdatestr.append('')
                    doccountstr.append('')
                    docfinestr.append('')

            se.w2vDocRank[idx]=w2vDocRank_full
            se.w2vTopSent[idx]=w2vTopSent_full
            se.RegNonReg[idx]=RegNoReg
            se.NewsSource[idx]=News
            se.source[idx] = sources
            se.textsummary[idx]=textsummary_full
            se.lastsearchdate[idx]=dates
         
            se.DocTitle[idx]=titles
            se.DocSentStr[idx]=docsentstr
            se.DocSimStr[idx]=docsimstr
            se.DocDateStr[idx]=docdatestr
            se.DocCountStr[idx]=doccountstr
            se.DocFineStr[idx]=docfinestr
            
            
            
            
            whole_list = list(zip(se.w2vDocRank[idx], se.urllist[idx],se.w2vTopSent[idx]
                                  , se.list_fuzzyscore[idx],se.list_matchedstems[idx]
                                  , se.list_fuzzyscoredetail[idx], se.textlist[idx]
                                  , se.RegNonReg[idx],se.NewsSource[idx],se.textsummary[idx]
                                  , se.lastsearchdate[idx],se.source[idx], se.DocTitle[idx]
                                  , se.DocSentStr[idx],se.DocSimStr[idx],se.DocDateStr[idx]
                                  , se.DocCountStr[idx],se.DocFineStr[idx]
                                 ))
            sorted_list = sorted(whole_list, key = operator.itemgetter(0), reverse = True)
            transposed_sorted = list(zip(*sorted_list))
            #print(transposed_sorted)
            #can be used to rerank and score docs diff
            LSA_filter_count = len([x for x in transposed_sorted[0] if x >= LSA_filter])
            se.LSA_filter_count[idx] = LSA_filter_count
            ## Extract the links and scores
            if LSA_filter_count > 0:
                se.w2vDocRank[idx] = transposed_sorted[0][0:LSA_filter_count]
                se.urllist[idx] = transposed_sorted[1][0:LSA_filter_count]
                se.w2vTopSent[idx] = transposed_sorted[2][0:LSA_filter_count]
                se.list_fuzzyscore[idx] = transposed_sorted[3][0:LSA_filter_count]
                se.list_matchedstems[idx] = transposed_sorted[4][0:LSA_filter_count]
                se.list_fuzzyscoredetail[idx] = transposed_sorted[5][0:LSA_filter_count]
                se.textlist[idx] = transposed_sorted[6][0:LSA_filter_count]
                se.RegNonReg[idx] = transposed_sorted[7][0:LSA_filter_count]
                se.NewsSource[idx] = transposed_sorted[8][0:LSA_filter_count]
                se.textsummary[idx] = transposed_sorted[9][0:LSA_filter_count]
                se.lastsearchdate[idx] = transposed_sorted[10][0:LSA_filter_count]
                se.source[idx] = transposed_sorted[11][0:LSA_filter_count]
                se.DocTitle[idx] = transposed_sorted[12][0:LSA_filter_count]
                
                se.DocSentStr[idx]= transposed_sorted[13][0:LSA_filter_count]
                se.DocSimStr[idx]= transposed_sorted[14][0:LSA_filter_count]
                se.DocDateStr[idx]= transposed_sorted[15][0:LSA_filter_count]
                se.DocCountStr[idx]= transposed_sorted[16][0:LSA_filter_count]
                se.DocFineStr[idx]= transposed_sorted[17][0:LSA_filter_count]

            else:
                se.w2vDocRank[idx] = []
                se.urllist[idx] = []
                se.w2vTopSent[idx] = []
                se.list_fuzzyscore[idx] = []
                se.list_matchedstems[idx] = []
                se.list_fuzzyscoredetail[idx] = []
                se.textlist[idx] = []
                se.RegNonReg[idx] =[] 
                se.NewsSource[idx] =[]
                se.textsummary[idx] =[]
                se.lastsearchdate[idx] =[]
                se.source[idx] = []
                se.DocTitle[idx]=[]
                se.DocSentStr[idx]= []
                se.DocSimStr[idx]= []
                se.DocDateStr[idx]= []
                se.DocCountStr[idx]= []
                se.DocFineStr[idx]=[] 
                
            print(se.urllist[idx])
            print('=====')
            print(transposed_sorted[7])
            print('NEW URL LIST LENGTH:',len(se.urllist[idx]))
    return se

####  se object class 

In [4]:
class SE():
    def __init__(self, search_lang = ['en'], searchid = '', fname='', lname=''
                 , mname='', city='', state='', state2='', country='', occupation='', company=''
                 , company2='',querylist=[],searchtermlist=[]):
    #Attributes about the entity
        self.entity_id = str(searchid)
        self.entity_fname = fname
        self.entity_lname = lname
        self.entity_mname = mname
        self.entity_city = city
        self.entity_state = state
        self.entity_state2 = state2
        self.entity_country = country
        self.entity_occupation = occupation
        self.entity_company = company
        self.entity_company2 = company2
    #Attributes about what we are searching for
        self.entity_querylist = querylist
        self.searchtermlist = searchtermlist
        self.search_lang = search_lang
        self.search_lang_ct = len(self.search_lang)
        self.search_lang_short = ['']*self.search_lang_ct
    #Attributes about returned result
        self.querylisturl = [[]]*self.search_lang_ct
        self.urllist = [[]]*self.search_lang_ct
        self.origtextlist =  [[]]*self.search_lang_ct#[{} for x in range(0,self.search_lang_ct)]
        self.lastsearchdate =  [[]]*self.search_lang_ct#[{} for x in range(0,self.search_lang_ct)] #zjc:7/19
        self.textsummary =  [[]]*self.search_lang_ct#[{} for x in range(0,self.search_lang_ct)] #zjc:7/19
        self.textlist = [[]]*self.search_lang_ct
        self.pdftext = [[]]*self.search_lang_ct
        self.static_search_result = ''
    #Attributes about matching
        self.list_fuzzyscore = [[]]*self.search_lang_ct
        self.list_fuzzyscoredetail = [[]]*self.search_lang_ct
        self.list_matchedstems = [[]]*self.search_lang_ct
    #Additional Attributes
        self.LSA_score = [[]]*self.search_lang_ct
        self.w2vInfo =  [[]]*self.search_lang_ct
        self.w2vDfSorted=  [[]]*self.search_lang_ct
        self.w2vDocRank=  [[]]*self.search_lang_ct
        self.w2vTopSent=  [[]]*self.search_lang_ct
        self.RegNonReg=  [[]]*self.search_lang_ct
        self.source = [[]]*self.search_lang_ct
        self.NewsSource=  [[]]*self.search_lang_ct
        self.DocCategory= [[]]*self.search_lang_ct
        self.Perc = [[]]*self.search_lang_ct
        self.DocTitle = [[]]*self.search_lang_ct
        
    #Final Scores
        self.LSA_filter_count = ['']*self.search_lang_ct
        self.riskscore_final = ['']*self.search_lang_ct
        self.link_score_threshold = ['']*self.search_lang_ct
        self.name_fuzzy_all = ['']*self.search_lang_ct
        self.location_fuzzy_all = ['']*self.search_lang_ct
        self.employment_fuzzy_all = ['']*self.search_lang_ct
    
    #new data 
        self.entitycount =[[]]*self.search_lang_ct
        self.entityfines = [[]]*self.search_lang_ct
        self.DocSentStr=[[]]*self.search_lang_ct
        self.DocSimStr=[[]]*self.search_lang_ct
        self.DocDateStr=[[]]*self.search_lang_ct
        self.DocCountStr=[[]]*self.search_lang_ct
        self.DocFineStr=[[]]*self.search_lang_ct

            
    def aggregate_info(self):
        printlist = [self.entity_fname, self.entity_lname, self.entity_mname, self.entity_city, self.entity_state, self.entity_state2, self.entity_country, self.entity_occupation, self.entity_company, self.entity_company2]
        return " ".join([x.lower() for x in printlist if x != ''])

    def location_provided(self):
        loclist = [self.entity_city, self.entity_state, self.entity_state2, self.entity_country]
        returnval = False
        if len([x for x in loclist if x != ''])>0: returnval = True
        return returnval

    def employment_provided(self):
        loclist = [self.entity_occupation, self.entity_company, self.entity_company2]
        returnval = False
        if len([x for x in loclist if x != ''])>0: returnval = True
        return returnval

    def to_json(self, location):
        #try:
        outdata = {}
        outdata['id'] = self.entity_id
        outdata['entity_fname'] = self.entity_fname
        outdata['entity_lname'] = self.entity_lname
        outdata['entity_mname'] = self.entity_mname
        outdata['entity_city'] = self.entity_city
        outdata['entity_state'] = self.entity_state
        outdata['entity_state2'] = self.entity_state2
        outdata['entity_country'] = self.entity_country
        outdata['entity_occupation'] = self.entity_occupation
        outdata['entity_company'] = self.entity_company
        outdata['entity_company2'] = self.entity_company2
        outdata['entity_querylist']=self.entity_querylist 
        outdata['searchtermlist']=self.searchtermlist 
        outdata['search_lang']=self.search_lang 
        outdata['search_lang_ct']=self.search_lang_ct 
        outdata['search_lang_short']=self.search_lang_short 
        outdata['querylisturl']=self.querylisturl 
        outdata['urllist']=self.urllist 
        outdata['origtextlist']=self.origtextlist
        outdata['textsummary']=self.textsummary 
        outdata['lastsearchdate'] = self.lastsearchdate 
        outdata['textlist']=self.textlist 
        outdata['pdftext']=self.pdftext 
        outdata['static_search_result']=self.static_search_result 
        outdata['list_fuzzyscore']=self.list_fuzzyscore 
        outdata['list_fuzzyscoredetail']=self.list_fuzzyscoredetail 
        outdata['list_matchedstems']=self.list_matchedstems 
        outdata['LSA_score']=self.LSA_score 
        outdata['LSA_filter_count']=self.LSA_filter_count 
        outdata['w2vInfo']=self.w2vInfo 
        outdata['w2vDocRank'] = self.w2vDocRank
        outdata['w2vTopSent'] = self.w2vTopSent
        outdata['textsummary'] = self.textsummary
        outdata['RegNonReg'] = self.RegNonReg
        outdata['source ']  = self.source 
        outdata['NewsSource'] = self.NewsSource
        outdata['DocCategory'] = self.DocCategory
        outdata['Perc'] = self.Perc
        outdata['DocTitle'] = self.DocTitle
        outdata['w2vDfSorted']=[x.to_json(orient='index') for x in self.w2vDfSorted]
        outdata['riskscore_final']=self.riskscore_final 
        outdata['link_score_threshold']=self.link_score_threshold 
        outdata['name_fuzzy_all']=self.name_fuzzy_all 
        outdata['location_fuzzy_all']=self.location_fuzzy_all 
        outdata['employment_fuzzy_all']=self.employment_fuzzy_all
        outdata['entitycount']=self.entitycount
        outdata['entityfines']=self.entityfines 
        outdata['DocSentStr']=self.DocSentStr
        outdata['DocSimStr']=self.DocSimStr
        outdata['DocDateStr']=self.DocDateStr
        outdata['DocCountStr']=self.DocCountSTr
        outdata['DocFineStr']=self.DocFineStr


        with open(location, 'w') as outfile:
            json.dump(outdata, outfile)
        return 0
        #except:
        #    return -1

#### Helper Functions

In [3]:
def Doccats_SummaryScores(se):
    #assign doc category 
    summary_scores(se, config.search_results_maxoutput)
    DocCat,Per = stemsims(se,wv)
    se.DocCategory[0] = DocCat
    se.Perc[0] = Per
    return se 


def stem_phrase(stems,wv):   
    z=[]
    for s in set(stems):
        x= np.zeros(300)
        if s in wv:
            x+=wv.query(s)
            #norm = preprocessing.normalize(x.reshape(1,-1))
            z.append(x)
    print(len(z))
    return z 
    

def avg_phrase(phrase, wv, z):
  
    v = np.zeros(300) #create empty vector to hold the average vector of the nword phrase being processed(likley a sentence)
    #for each word that is in the sentence being processed 
    #note: we use set() operator here to remove duplicate values of words, we only want to capture each word in the phrase one time when calculating average word vectors. 
    sim_list = []
            
    for w in phrase: #set(phrase):
       #if the word is found in the google word embeddings 
        if w in wv:
          #extract the token index associated with the word from the dictonary of all terms contained in the tf-idf 
          #  word_id = dictionary.token2id[w]
          #extract the tfidf vector of weightings for the document being processed
          #  doc_vectors = tf_idf[doc_id]
          #create a dictonary from the resulting doc_vector (word_index, tf-idf weighting) 
          #  d = dict(doc_vectors)
          #return the tf-idf from the dictonary associated with the tf-idf vector of weightings for the doc being processed and extract the weight associated with the word being processed
          #  tf_score = d[word_id] 
          #alter the google word vector representing the word being procssed to account for its importance in the document being processed via tf-idf weighting 
            sim=wv.query(w)
            #calculate avg vector of words from the sentence if theyre cor with the risk term is over a specified threshold 
            for vec in z:
                #simm= cosine_sim(preprocessing.normalize(sim.reshape(1,-1)), vec[0])
                simm= cosine_sim(sim, vec)
                #print(simm)
                if simm > .1:
                    sim_list.append(simm)
                #if cosine_sim(preprocessing.normalize(sim.reshape(1,-1)), vec) > .1:
#                     v += wv.query(w)
    if len(sim_list) != 0:
        sim_av = sum(sim_list)/len(sim_list)
    else:
        sim_av = 0
    sim_num = len(sim_list)
                    
   #after all words in the sentence have been processed normilize the final vector back to unit length reprsenting the average word embedding vector for the terms in the sentence 
    #v_norm = preprocessing.normalize(v.reshape(1,-1))
    print(sim_av)
    return  sim_av, sim_num#v_norm


# Function to get cosine similarity 
def cosine_sim(a, b):
    return 1 - spatial.distance.cosine(a, b)

def validate_date(date_text,date_form="%m-%d-%Y"):
    try:
        date_text2=dateparser.parse(str(date_text)).strftime("%m-%d-%Y")
        print('sucesss')
    except Exception: 
        return False
    if date_text2 != datetime.datetime.strptime(dateparser.parse(str(date_text)).strftime("%m-%d-%Y"), date_form).strftime(date_form):
        print('fail')
        return False
    else: 
        print('sucesss')
        return True
    
def asymptoc(x):
    x = (.3*x**2) / ((.3*x**2)+2)
    return x

#weight_func(x.DocSim,-1*x.DocSent,x.DocDateN,x.entityWeight,x.fineWeight), axis = 1)
def weight_func(x,y,z,a,b):
    #print((x * 0.5) + (y * 0.5))
    return ((x * 0.8) + (y * 0.01) + (z * .01) + (a*.08) + (b*.1)) 

def rough_date_pandas_wrapper(row):
    '''Wrapper function that searches for a year value in each column of the passed the 
    pandas DataFrame row via applying the rough_date function as described below
    
    Arguments:
        row {pandas.Series} -- a pandas DataFrame row in which to search for a date value
    
    Returns:
        int -- first year value found from applying the rough_date function to the columns from left to right

    Usage:
        df['year'] = df.appy(rough_date_pandas_wrapper, axis = 1)
    '''

    NO_YEAR_FOUND = 0
    def rough_date(text):
        '''Function to return the first year value as an integer from the passed string

        Arguments:
            text {str} -- the text through which to sort 
        
        Returns:
            int -- the first year value from the passed string

        Usage:
            rough_date("This was written in 2009.") --> 2009
            rough_date("He was born in 1998 but his brother was born in 1993.") --> 1998
            rough_date("He makes more than $2000 per week.") --> 0
            rough_date("This sentence does not contain any numbers.") --> 0
        '''

        YEAR = re.compile(r'\d?\d[//-]\d?\d[//-](\d+)|[^\$](20[0|1|2]\d)|[^\$](19\d\d)')

        def extract_first_regex_result(regex_result):
            '''Utility function to return the first non-blank result from a regex search result tuple
            
            Arguments:
                regex_result {list} -- list containing a single tuple of strings through which to search for a non-blank value
            
            Returns:
                any -- the first non-blank member of the first tuple in the passed tuple list

            Usage:
                extract_first_regex_result( [('', '2018', '')] ) --> '2018'
                extract_first_regex_result( [('2017', '2018', '')] ) --> '2017'
                extract_first_regex_result( [('','','')] ) --> None
            '''

            if regex_result == []:
                return
            regex_result = regex_result[0]
            for i in regex_result:
                if i is not '':
                    return i

        def format_year(year):
            '''Utility function to format strings representing years as integers
            
            Arguments:
                year {str} -- string representing year to format
            
            Returns:
                int -- year as integer

            Usage:
                format_year("99") --> 1999
                format_year("0") --> 2000
                
                format_year("1999") --> 1999
                format_year("2000") --> 2000

                format_year("fail") --> 0
            '''

            if not isinstance(year, int):
                try:
                    year = int(year)
                except Exception:
                    return NO_YEAR_FOUND

            if year < 100:
                year += 2000

            return year
        
        try:
            text = str(text)
        except Exception:
            text = ''

        return format_year(extract_first_regex_result(YEAR.findall(text)))
    
    for col in row:
        date = rough_date(col)
        if date != NO_YEAR_FOUND:
            return date
    return NO_YEAR_FOUND

In [2]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
import os
import sys 
#comment for build 4
#comment again 2
from nn_base import nn_extractfines as ef
try:
    entity_dict = ef.load_entity_dict('entity_dict.pickle')
except FileNotFoundError:
    entity_dict = ef.load_entity_dict('resources/entity_dict.pickle')

import spacy
nlp = spacy.load('en_core_web_sm')
from functools import reduce

import errno
from nn_config import * 
import time
import html
import pdb
import cgi
import html
#from nn_base.nn_classes import search_entity as SE
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, PageBreak
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfFileMerger
from docx import Document
import docx
from docx.shared import RGBColor


def create_pdf_wrapper(se):
    #fname: filename of pdf created , failed: list of languages that failed to create a pdf 
    #fnames = []
    fname, failed = create_pdf_summary(se)

    print('PDF file stored at - {}'.format(os.path.basename(fname)))
    for x in failed:
        #fnames.append((x,time.strftime("%Y-%m-%d-%H-%M-%S"), 999999))
        fnames=str(x.replace(' ','')+'_'+time.strftime("%Y-%m-%d-%H-%M-%S")+'_'+str(999999))
        print('PDF creation failed on language - {}'.format(x))
    else: 
        fnames=str(fname.replace(' ','')+'_'+time.strftime("%Y-%m-%d-%H-%M-%S")+'_'+str(se.riskscore_final[0]))

    jsonname = os.path.splitext(fname)[0]+'.json'

    try:
        dataout = se.to_json(jsonname)
    except Exception:
        print('exception data not saved for given file path {}',jsonname)
        dataout=12
    pass
    if dataout == 0: #to_json() will return a 0 if sucsesful 
        print('Data saved at {}'.format(os.path.basename(jsonname)))
    else: 
        print('Data Not Saved')
    #print(fnames)
    return fnames

def remove_if_exist(filename):
    try:
        os.remove(filename)
    except OSError as e:
        raise e
        
def create_pdf_summary(se):
    
    fnames_list = [] ##
    pdf_path = config.pdf_path
    #final_fname = pdf_path + str(se.entity_id) + '_' + se.entity_fname + ' ' + se.entity_lname+ ' ' + time.strftime("%Y-%m-%d-%H-%M-%S") + '.pdf'
    finalW_fname = pdf_path + str(se.entity_id) + '_' + se.entity_fname + ' ' + se.entity_lname+ ' ' + time.strftime("%Y-%m-%d-%H-%M-%S") + '.docx'

    for idx in range(0,se.search_lang_ct):
        if se.urllist[idx] == []:
            fname,word = create_empty_page(se, idx, pdf_path)
            fnames_list.append(fname)
        else:
#             try:
            fname,word = create_nn_page(se, idx, pdf_path)
            fnames_list.append(fname)
#             except:
#                 fname,word = create_empty_page(se,idx, pdf_path)
    # merger = PdfFileMerger()
    # toremove=[]
    failed = []
    # for idx, pdf in enumerate(fnames_list):
    #     if pdf != -1:
    #         temp = open(pdf,'rb')
    #         merger.append(temp)
    #         toremove.append(temp)
    #     else:
    #         failed.append(se.search_lang[idx])
    # with open(final_fname, 'wb') as finalout:
    #     merger.write(finalout)
    # for pdf in toremove:
    #     pdf.close()
    # for pdf in fnames_list:
    #     if pdf!= -1:
    #         remove_if_exist(pdf)
    word.save(finalW_fname)
    return fname, failed
          
def create_empty_page(se, idx, pdf_path):
    pdf_fname = pdf_path + str(se.entity_id) + '_' + se.entity_fname + ' ' + se.entity_lname+ ' ' + time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + str(idx) + '.pdf'
    styles = getSampleStyleSheet()
    doc = SimpleDocTemplate(pdf_fname, leftMargin = 0.75*inch, rightMargin = 0.75*inch, topMargin = 1*inch, bottomMargin = 1*inch)
    Story=[] 
    h1_1=Paragraph(se.entity_fname + ' ' + se.entity_lname + ' - ' + se.search_lang[idx][0].upper() + se.search_lang[idx][1:], styles['Title'])
    Story.append(h1_1)
    h2_1=Paragraph("Search link", styles['Heading2'])
    Story.append(h2_1)
    print(se.querylisturl[idx])
    #p_1 = Paragraph(cgi.escape(se.querylisturl[idx].replace(' ','%20')), styles['Normal'])    
    #Story.append(p_1)
    h2_2=Paragraph("No negative links found", styles['Heading2'])
    Story.append(h2_2)
    doc.build(Story)

    #Word doc
    word_doc = Document()
    p = word_doc.add_heading('Entity Searched: ' + str(se.entity_fname), level = 0)
    p = word_doc.add_heading('No negative links found') 
    
    return pdf_fname,word_doc

def create_empty_page_no_se(entity_name):
    fnames_list = []
    pdf_fname = pdf_path + str(entity_name) + time.strftime("%m-%d-%Y") + '.pdf'
    styles = getSampleStyleSheet()
    doc = SimpleDocTemplate(pdf_fname, leftMargin = 0.75*inch, rightMargin = 0.75*inch, topMargin = 1*inch, bottomMargin = 1*inch)
    Story=[] 
    h1_1=Paragraph(str(entity_name), styles['Title'])
    Story.append(h1_1)
    # h2_1=Paragraph("Search link", styles['Heading2'])
    # Story.append(h2_1)
    # print(se.querylisturl[idx])
    #p_1 = Paragraph(cgi.escape(se.querylisturl[idx].replace(' ','%20')), styles['Normal'])    
    #Story.append(p_1)
    h2_2=Paragraph("No negative links found", styles['Heading2'])
    Story.append(h2_2)
    doc.build(Story)

    #Word doc
    word_doc = Document()
    p = word_doc.add_heading('Entity Searched: ' + str(entity_name), level = 0)
    p = word_doc.add_heading('No negative links found') 
    fnames_list.append(fname)
    return pdf_fname,word_doc

    
def create_nn_page(se, idx, pdf_path):
    final_links = se.urllist[idx]
    max_links_for_score = se.LSA_filter_count[idx]
    links_after_filter = len(se.urllist[idx])
    
    if links_after_filter >  search_results:
        max_links_for_score = search_results #search_results_maxoutput
        
    #avg_identity_scores = se.list_fuzzyscore[idx]
    #matched_stems_list  = se.list_matchedstems[idx]
    entity_fname = se.entity_fname
    #non_reg = se.RegNonReg
    normalized_risk_score =min(100,int(se.riskscore_final[idx])) 
    name_fuzzy_score = se.name_fuzzy_all[idx]
    location_fuzzy_score = se.location_fuzzy_all[idx]
    employment_fuzzy_score = se.employment_fuzzy_all[idx]
    #queryurl = se.querylisturl[idx]
    w2vtopsent = se.w2vTopSent[idx]
    summary = se.textsummary[idx]
    regnoreg = se.RegNonReg[idx]
    docCat = se.DocCategory[idx]
    percC = se.Perc[idx]
    newssource = se.NewsSource[idx]
    datelist = se.lastsearchdate[idx]
    docTitles = se.DocTitle[idx]
    NumStartUrls = len(se.origtextlist[idx])
    textsumm = se.textsummary[idx]
    
    #new for dtcc 
    FinalDocScore = se.w2vDocRank[idx]
    SimRank = se.DocSimStr[idx]
    SentRank = se.DocSentStr[idx]
    DateRank = se.DocDateStr[idx]
    FineRank= se.DocFineStr[idx]
    EntityRank = se.DocCountStr[idx]
    
    
    #Compute regulatory percentages
    from collections import Counter
    c = Counter(regnoreg)
    oo = [(i, c[i] / len(regnoreg) * 100.0) for i in c]
    if len(oo)>1:
        Reg = ([item[1] for item in oo if item[0] == "Regulatory"][0])
        NoReg= ([item[1] for item in oo if item[0] == "Non Regulatory"][0])
        PerReg = 100*(Reg/(Reg+NoReg))
    elif oo[0][0] == 'Non Regulatory':
        PerReg = 0
    else:
        PerReg = 100
    
    #Counters for article types
    regCount = 0.01*PerReg*len(summary)
    MFcount = sum(docCat[0])
    LAcount = sum(docCat[2])
    RRcount = sum(docCat[3])

    styles = getSampleStyleSheet()
    outfn = pdf_path + se.entity_fname + ' ' + se.entity_lname+ ' ' + time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + str(idx) + '.pdf'
    ##WORD
    word_doc = Document()
    
    #try:
    pdf_merge_list =[]
    ## Overview pdf generations
    #pdf_filename = pdf_path + se.entity_fname + '_'+se.entity_lname + '_Overview.pdf'
    pdf_filename = pdf_path + str(se.entity_id) + '_' + se.entity_fname + ' ' + se.entity_lname+ ' ' + time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + str(idx) + '.pdf'
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter, leftMargin = 0.75*inch, rightMargin = 0.75*inch, topMargin = 1*inch, bottomMargin = 2*inch)
    Story=[] 
    h1_1=Paragraph('Entity Searched: ' + se.entity_fname + ' ' + se.entity_lname, styles['Title'])
    p = word_doc.add_heading('Entity Searched: ' + str(se.entity_fname), level = 0)
    Story.append(h1_1)
    p1=Paragraph("<para align=center><br/><font size=11><b><u>Total Negative News Score</u></b></font><font size=8><sup>1</sup></font>: " + str(round(normalized_risk_score,2)) + "%" + "</para>", styles['Heading2']) 
    Story.append(p1)
    p = word_doc.add_heading('Total Negative News Score: ')
    p.add_run(str(round(normalized_risk_score,2)) + "%")
   
    if se.static_search_result != '':
        totallist = [item for searchlist in [list(x[1].keys()) for x in se.static_search_result.items() if len(x[1].keys())>0] for item in searchlist]
        px = Paragraph("<para align=center><font size=10><b><u>Warning: Name Found on targeted search list:</u></b></font></para>", styles['Heading2'])
        Story.append(px)
        for res in totallist:
            px = Paragraph("<para lindent=10><font size=9>"+res+ "</font></para>", styles['Normal'])
            Story.append(px)

    t_h=Paragraph("<para align=center><br/><font size=11><b><u>Details on Links Searched</u></b></font><font size=8><sup>2</sup></font> </para>", styles['Heading2'])
    Story.append(t_h)  
    
    p = word_doc.add_heading("Details on Links Searched", level = 1)
    tableW = word_doc.add_table(rows = 3, cols = 2)
    tableW.style = 'TableGrid'
    cell1 = tableW.cell(0,0)
    cell1.text = 'Links returned'
    cell2 = tableW.cell(0,1)
    cell2.text = str(NumStartUrls)
    cell3 = tableW.cell(1,0)
    cell3.text = 'Remaining after filtering'
    cell4 = tableW.cell(1,1)
    cell4.text = str(links_after_filter)
    cell5 = tableW.cell(2,0)
    cell5.text = 'Negative links used for risk scoring'
    cell6 = tableW.cell(2,1)
    cell6.text = str(max_links_for_score)
    
    p = word_doc.add_heading("Average Relevance Scores", level = 1)
    tableW = word_doc.add_table(rows = 3, cols = 2)
    tableW.style = 'TableGrid'
    cell1 = tableW.cell(0,0)
    cell1.text = 'Name'
    cell2 = tableW.cell(0,1)
    cell2.text = str(name_fuzzy_score) + "%"
    cell3 = tableW.cell(1,0)
    cell3.text = 'Location'
    cell4 = tableW.cell(1,1)
    cell4.text = str(location_fuzzy_score)
    cell5 = tableW.cell(2,0)
    cell5.text = 'Employment'
    cell6 = tableW.cell(2,1)
    cell6.text = str(employment_fuzzy_score)
    
    p = word_doc.add_heading("Regulatory Site Count", level = 1)
    tableW = word_doc.add_table(rows = 2, cols = 3)
    tableW.style = 'TableGrid'
    cell1 = tableW.cell(1,0)
    cell1.text = 'Regulatory Site Articles'
    cell2 = tableW.cell(1,2)
    cell2.text = str(round(PerReg,1)) + "%"
    cell3 = tableW.cell(0,1)
    cell3.text = 'Count'
    cell4 = tableW.cell(1,1)
    cell4.text = str(int(round(regCount,1)))
    cell7 = tableW.cell(0,2)
    cell7.text = 'Percent'
    
    p = word_doc.add_heading("Article type summary", level = 1)
    tableW = word_doc.add_table(rows = 4, cols = 3)
    tableW.style = 'TableGrid'
    cell1 = tableW.cell(0,1)
    cell1.text = 'Count'
    cell2 = tableW.cell(0,2)
    cell2.text = 'Percent'
    cell3 = tableW.cell(1,0)
    cell3.text = 'Monetary Fine'
    cell33 = tableW.cell(1,1)
    cell33.text = str(MFcount)
    cell4 = tableW.cell(1,2)
    cell4.text = str(round(100*percC[0],1)) + "%"
    cell5 = tableW.cell(2,0)
    cell5.text = 'Legal Action'
    cell55 = tableW.cell(2,1)
    cell55.text = str(LAcount)
    cell6 = tableW.cell(2,2)
    cell6.text = str(round(100*percC[2],1)) + "%"
    cell7 = tableW.cell(3,0)
    cell7.text = 'Regulatory References'
    cell77 = tableW.cell(3,1)
    cell77.text = str(RRcount)
    cell8 = tableW.cell(3,2)
    cell8.text = str(round(100*percC[3],1)) + "%"
    
    #Model weight table first page Word doc
    p = word_doc.add_heading("Document Ranking Weights", level = 1)
    tableW = word_doc.add_table(rows = 2, cols = 5)
    tableW.style = 'TableGrid'
    cell1 = tableW.cell(0,0)
    cell1.text = 'Similarity'
    cell2 = tableW.cell(0,1)
    cell2.text = 'Sentiment'
    cell3 = tableW.cell(0,2)
    cell3.text = 'Date'
    cell4 = tableW.cell(0,3)
    cell4.text = 'Entity Count'
    cell5 = tableW.cell(0,4)
    cell5.text = 'Fine'
    cell11 = tableW.cell(1,0)
    cell11.text = str(model_weights[0])
    cell22 = tableW.cell(1,1)
    cell22.text = str(model_weights[1])
    cell33 = tableW.cell(1,2)
    cell33.text = str(model_weights[2])
    cell44 = tableW.cell(1,3)
    cell44.text = str(model_weights[3])
    cell55 = tableW.cell(1,4)
    cell55.text = str(model_weights[4])
    
    p = word_doc.add_heading("Domains Searched", level = 1)
    p = word_doc.add_paragraph(" ")
    p.add_run(str(se.entity_querylist))

    p = word_doc.add_heading("Terms Used In Analysis", level = 1)
    p = word_doc.add_paragraph(" ")
    p.add_run(str(searchterms1))
    
    word_doc.add_page_break()
    
    
    row1 = Paragraph("<para align=center>Links returned </para>", styles['Normal'])
    row2 = Paragraph("<para align=center>Remaining after filtering</para>", styles['Normal'])
    row3 = Paragraph("<para align=center>Negative links used for risk scoring</para>", styles['Normal'])
    row1_2 = Paragraph('<para align=center>' + str(NumStartUrls) + '</para>', styles['Normal'])
    row2_2 = Paragraph('<para align=center>' + str(links_after_filter) + '</para>', styles['Normal'])
    row3_2 = Paragraph('<para align=center>' + str(max_links_for_score) + '</para>', styles['Normal'])
    data= [[row1, row1_2],
           [row2, row2_2],
           [row3,row3_2]]
    t=Table(data, hAlign='CENTER')
    t.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                          ('BOX', (0,0), (-1,-1), 0.25, colors.black)]))
    Story.append(t)
    p3=Paragraph("<para align=center><font size=11><b><u>Average Relevance Scores</u></b></font><font size=8><sup>3</sup></font> </para>", styles['Heading2'])
    Story.append(p3)
    p4_1 = Paragraph("<para align=center>Name</para>",styles['Normal'])
    p4_2 = Paragraph("<para align=center>Location</para>",styles['Normal'])
    p4_3 = Paragraph("<para align=center>Employment</para>",styles['Normal'])
    p5_1 = Paragraph("<para align=center>" + str(name_fuzzy_score) + '</para>', styles['Normal'])
    p5_2 = Paragraph("<para align=center>" + str(location_fuzzy_score) + '</para>', styles['Normal'])
    p5_3 = Paragraph("<para align=center>" + str(employment_fuzzy_score) + '</para>', styles['Normal'])
    data2= [[p4_1, p5_1],
            [p4_2, p5_2],
            [p4_3, p5_3]]
    t2=Table(data2, hAlign='CENTER')
    t2.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black)]))
    Story.append(t2)
    
    p5=Paragraph("<para align=center><font size=11><b><u>Regulatory Sites</u></b></font><font size=8><sup>3</sup></font> </para>", styles['Heading2'])
    Story.append(p5)
    p4_1 = Paragraph("<para align=center>Regulatory Site Articles</para>",styles['Normal'])
    p4_2 = Paragraph("<para align=center>Count</para>",styles['Normal'])
    p4_3 = Paragraph("<para align=center>Percent</para>",styles['Normal'])
    p5_2 = Paragraph("<para align=center>" + str(round(PerReg,2)) + "%" + '</para>', styles['Normal'])
    p5_1 = Paragraph("<para align=center>" + str(int(round(regCount,1))) + '</para>', styles['Normal'])
    p5_3 = Paragraph("<para align=center> </para>",styles['Normal'])
    data2= [[p5_3, p4_2,p4_3],
            [p4_1, p5_1, p5_2]]
    t2=Table(data2, hAlign='CENTER')
    t2.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black)]))
    Story.append(t2)
    
    p4=Paragraph("<para align=center><font size=11><b><u>Article Type Summary</u></b></font> </para>", styles['Heading2'])
    Story.append(p4)
    p0_1 = Paragraph("<para align=center> </para>",styles['Normal'])
    p0_2 = Paragraph("<para align=center>Count</para>",styles['Normal'])
    p0_3 = Paragraph("<para align=center>Percent</para>",styles['Normal'])
    p4_1 = Paragraph("<para align=center>Monetary Fine</para>",styles['Normal'])
    p4_2 = Paragraph("<para align=center>Legal Action</para>",styles['Normal'])
    p4_3 = Paragraph("<para align=center>Regulatory Reference</para>",styles['Normal'])
    #p5_3 = Paragraph("<para align=center>" + str(round(PerReg,2)) + "%" + '</para>', styles['Normal'])
    p5_1 = Paragraph("<para align=center>" + str(round(100*percC[0],1)) + "%" + '</para>', styles['Normal'])
    p5_2 = Paragraph("<para align=center>" + str(round(100*percC[2],1)) + "%" + '</para>', styles['Normal'])
    p5_3 = Paragraph("<para align=center>" + str(round(100*percC[3],1)) + "%" + '</para>', styles['Normal'])
    p6_1 = Paragraph("<para align=center>" + str(MFcount) + '</para>', styles['Normal'])
    p6_2 = Paragraph("<para align=center>" + str(LAcount) + '</para>', styles['Normal'])
    p6_3 = Paragraph("<para align=center>" + str(RRcount) + '</para>', styles['Normal'])
    data2= [[p0_1, p0_2, p0_3],
            [p4_1, p6_1, p5_1],
            [p4_2, p6_2, p5_2],
            [p4_3, p6_3, p5_3]]
    t2=Table(data2, hAlign='CENTER')
    t2.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black)]))
    Story.append(t2)
    
    #Model weight table first page PDF
    p312=Paragraph("<para align=center><font size=11><b><u>Document Ranking Weightings</u></b></font><font size=8><sup>3</sup></font> </para>", styles['Heading2'])
    Story.append(p312)
    row1 = Paragraph("<para align=center>Similarity</para>", styles['Normal'])
    row2 = Paragraph("<para align=center>Sentiment</para>", styles['Normal'])
    row3 = Paragraph("<para align=center>Date</para>", styles['Normal'])
    row4 = Paragraph("<para align=center>Entity Count</para>", styles['Normal'])
    row5 = Paragraph("<para align=center>Fine</para>", styles['Normal'])
    row1_2 = Paragraph('<para align=center>' + str(model_weights[0]) + '</para>', styles['Normal'])
    row2_2 = Paragraph('<para align=center>' + str(model_weights[1]) + '</para>', styles['Normal'])
    row3_2 = Paragraph('<para align=center>' + str(model_weights[2]) + '</para>', styles['Normal'])
    row4_2 = Paragraph('<para align=center>' + str(model_weights[3]) + '</para>', styles['Normal'])
    row5_2 = Paragraph('<para align=center>' + str(model_weights[4]) + '</para>', styles['Normal'])
    data= [[row1, row2, row3, row4, row5],
           [row1_2, row2_2, row3_2, row4_2, row5_2]]
    t=Table(data, hAlign='CENTER')
    t.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                          ('BOX', (0,0), (-1,-1), 0.25, colors.black)]))
    Story.append(t)
    
    h2_1=Paragraph("<para align=center><font size=11><b><u>Domains Searched</u></b></font><font size=8><sup>4</sup></font> </para>", styles['Heading2'])
    Story.append(h2_1)
    address = '<para align=center><link href="' + '">' + se.entity_querylist + '</link></para>'
    a_1 = Paragraph(address, styles['Normal'])
    Story.append(a_1)
    h2_11=Paragraph("<para align=center><font size=11><b><u>Terms Used In Analysis</u></b></font><font size=8><sup>4</sup></font> </para>", styles['Heading2'])
    Story.append(h2_11)
    address2 = '<para align=center><link href="' + '">' + str(searchterms1) + '</link></para>'
    a_2 = Paragraph(address2, styles['Normal'])
    Story.append(a_2)
    #doc.multiBuild(Story, canvasmaker=FooterCanvas)
    #doc.build(Story)
    #pdf_merge_list.insert(0, pdf_filename)
    Story.append(PageBreak())

    ## Overview pdf generation
    #pdf_filename2 = pdf_path + se.entity_fname + '_'+se.entity_lname + '_Overview2.pdf'
    #doc = SimpleDocTemplate(pdf_filename2, pagesize=letter, leftMargin = 0.75*inch, rightMargin = 0.75*inch, topMargin = 1*inch, bottomMargin = 1*inch)
    #Story=[]         
    h2_4 = Paragraph("<para align=center><font size = 11><u>Links</u></font> </para>", styles['Heading2'])
    Story.append(h2_4)
    p = word_doc.add_heading('Links', level = 1)
    print('NewsSource: ' ,newssource)
    
    for t in range(0,max_links_for_score):  
        #reg vs not reg identifier
        tempR = Paragraph(str(t+1) + ". " + "<u>" + str(regnoreg[t]) + "</u>", styles['Normal'])
        Story.append(tempR)
        #p = word_doc.add_paragraph(str(t+1) + '. ')
        p = word_doc.add_paragraph("",style='List Number')
        p.add_run(str(regnoreg[t])).underline = True
        
        fine_reports = []
        fine_reports.extend(ef.generate_report_string(docTitles[t], entity_name = se.entity_fname, content = ' '.join(w2vtopsent[t]), nlp = nlp, entity_dict = entity_dict))
        fine_reports.extend([fr for sent in w2vtopsent[t] for fr in ef.generate_report_string(sent, entity_name = se.entity_fname, nlp = nlp, entity_dict = entity_dict)])
        fine_reports = set(fine_reports)

        DocCats = []
        if (docCat[0][t] == True) or (len(fine_reports) > 0):
            DocCats.append("Monetary Fine ")
        if docCat[2][t] == True:
            DocCats.append("Legal Action ")
        if docCat[3][t] == True:
            DocCats.append("Regulatory Reference ")
        if docCat[0][t]+docCat[2][t]+docCat[3][t] == 0:
            DocCats.append("Other Negative News")
            
        # Entity name
        tempE = Paragraph("<para lindent=11>" + "<u>" + str(entity_fname).upper() + "</u>"  , styles['Normal'])
        p = word_doc.add_paragraph(" ")
        p.add_run(str(entity_fname).upper()).underline = True
        Story.append(tempE)
        #Doc Cat
        tempA = Paragraph("<para lindent=11>" + str(' '.join(DocCats)), styles['Normal'])
        p = word_doc.add_paragraph(" ")
        p.add_run(str(' '.join(DocCats)))
        Story.append(tempA)
        #Add table for binned ranking values
        tableRk = word_doc.add_table(rows = 2, cols = 6)
        tableRk.style = 'TableGrid'
        cell1 = tableRk.cell(0,0)
        cell1.text = 'Stem Similarity'
        cell2 = tableRk.cell(0,1)
        cell2.text = 'Stem Sentiment'
        cell3 = tableRk.cell(0,2)
        cell3.text = 'Date'
        cell4 = tableRk.cell(0,3)
        cell4.text = 'Fine'
        cell5 = tableRk.cell(0,4)
        cell5.text = 'Entity found'
        cell6 = tableRk.cell(0,5)
        cell6.text = 'Article Rank'
        bolding_row = [0]
        for col in list(range(6)):
            for row in bolding_row:
                tableRk.rows[row].cells[col].paragraphs[0].runs[0].font.bold = True
        cell11 = tableRk.cell(1,0)
        cell11.text = str(SimRank[t])
        cell22 = tableRk.cell(1,1)
        cell22.text = str(SentRank[t])
        cell33 = tableRk.cell(1,2)
        cell33.text = str(DateRank[t])
        cell44 = tableRk.cell(1,3)
        cell44.text = str(FineRank[t])
        cell55 = tableRk.cell(1,4)
        cell55.text = str(EntityRank[t])
        cell66 = tableRk.cell(1,5)
        cell66.text = str(round(FinalDocScore[t],3))
     
        tempSumm = Paragraph("<para lindent=11> " + "<br/>" + " " + '</para>', styles['Normal']) 
        Story.append(tempSumm)
        row1 = Paragraph("<para align=center><b>Stem Similarity</b></para>", styles['Normal'])
        row2 = Paragraph("<para align=center><b>Stem Sentiment</b></para>", styles['Normal'])
        row3 = Paragraph("<para align=center><b>Date</b></para>", styles['Normal'])
        row4 = Paragraph("<para align=center><b>Fine</b></para>", styles['Normal'])
        row5 = Paragraph("<para align=center><b>Entity found</b></para>", styles['Normal'])
        row6 = Paragraph("<para align=center><b>Article Rank</b></para>", styles['Normal'])
        row1_2 = Paragraph('<para align=center>' + str(SimRank[t]) + '</para>', styles['Normal'])
        row2_2 = Paragraph('<para align=center>' + str(SentRank[t]) + '</para>', styles['Normal'])
        row3_2 = Paragraph('<para align=center>' + str(DateRank[t]) + '</para>', styles['Normal'])
        row4_2 = Paragraph('<para align=center>' + str(FineRank[t]) + '</para>', styles['Normal'])
        row5_2 = Paragraph('<para align=center>' + str(EntityRank[t]) + '</para>', styles['Normal'])
        row6_2 = Paragraph('<para align=center>' + str(round(FinalDocScore[t],3)) + '</para>', styles['Normal'])
        data= [[row1, row2,row3,row4,row5,row6],
               [row1_2,row2_2,row3_2,row4_2,row5_2,row6_2]]
        ttt=Table(data, hAlign='CENTER')
        ttt.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                              ('BOX', (0,0), (-1,-1), 0.25, colors.black)]))
        Story.append(ttt)
        
        #site name from se
#         print('NewsSoource: ', newssource[t])
#         print('FInal lins: ', final_links[t])
        p = word_doc.add_paragraph(" ")
        tempSumm = Paragraph("<para lindent=11> " + "<br/>" + " " + '</para>', styles['Normal']) 
        Story.append(tempSumm)
        tempN = Paragraph("<para lindent=11>"  + str(newssource[t]) + " Press Release: " + html.escape(final_links[t]) + "    ",styles['Normal'])
        p = word_doc.add_paragraph(" ")
        p.add_run(str(newssource[t]))
        p.add_run(" Press Release: ")
        hyperlink = add_hyperlink(p, html.escape(final_links[t]),html.escape(final_links[t]),'222AFF',False)
        #wp = p.add_run(cgi.escape(final_links[t]))
        #wp.italic = True
        #wp.font.color.rgb = RGBColor(0,0,255)
        Story.append(tempN)
        #Date published
        tempDate = Paragraph("<para lindent=11>" + '\n' + "<u>" + " " + str(datelist[t]) + " - " + str(docTitles[t]) + "</u>",styles['Normal'])
        p = word_doc.add_paragraph(" ")
        p.add_run('').underline = True
        p.add_run(str(datelist[t])).underline = True
        p.add_run(" - ")
        p.add_run(str(docTitles[t])).underline = True
        Story.append(tempDate)

        #Text summary
        tempSum = Paragraph("<para lindent=11> " + "<br/>" + "Article Summary: " + '</para>', styles['Normal']) 
        Story.append(tempSum)
        p = word_doc.add_paragraph(" ")
        p.add_run('Article Summary:').underline = True
        tempS = Paragraph("<para lindent=45> " + str(textsumm[t]) + "<br/>" + '</para>', styles['Normal']) 
        Story.append(tempS)
        p = word_doc.add_paragraph(str(textsumm[t]))

        #Top sentences
        tempx = Paragraph("<para lindent=11> " + "<br/>" + "Key Findings: " + '</para>', styles['Normal'])
        Story.append(tempx)
        p = word_doc.add_paragraph(" ")
        p.add_run('Key Findings:').underline = True
        for w in w2vtopsent[t]:
            p = word_doc.add_paragraph(w, style = 'List Bullet')
            
        for w in w2vtopsent[t]:
            tempx = Paragraph("<para lindent=45> "  + '\n\u2022' + str(w) + "<br/>" + '</para>', styles['Normal']) 
            Story.append(tempx)
        
        temp5 = Paragraph("<para><br/><br/></para>",styles['Normal'])
        if len(fine_reports) > 0:
            fine_pdf_info = reduce(lambda agg, x: agg + x + "<br/>", fine_reports, "<para lindent=11><u>Actions Taken:</u><br/>") + "<br/></para>"
            temp5 = Paragraph(fine_pdf_info,styles['Normal'])
            p = word_doc.add_paragraph(" ")
            p.add_run('Actions Taken: ').underline = True
            for fine_report in fine_reports:
                p = word_doc.add_paragraph(" ")
                p.add_run(fine_report)
        Story.append(temp5)

        temp4 = Paragraph("<para lindent=11>",styles['Normal'])
        Story.append(temp4)
        p = word_doc.add_paragraph(" ")
    doc.build(Story) 
    
    ##WORD
    word_doc.add_paragraph(Story)
    
    #pdf_merge_list.insert(1, pdf_filename2)
    ## Merging all pdfs to generate final
#     inputs = pdf_merge_list
#     merger = PdfFileMerger()
#     temppdfs = []
#     for pdf in inputs:
#         temp = open(pdf,'rb')
#         merger.append(temp)
#         temppdfs.append(temp)
#     with open(outfn, 'wb') as fout:
#         merger.write(fout)    
# #    merger.close()
#     for pdf in temppdfs:
#         pdf.close()
#     for pdf in inputs:
#         remove_if_exist(pdf)
    # return (outfn,word_doc)
    return (pdf_filename,word_doc)
    
    
### defining footer for pdf   
class FooterCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self.pages = []
    def showPage(self):
        self.pages.append(dict(self.__dict__))
        self._startPage()
    def save(self):
        page_count = len(self.pages)
        for page in self.pages:
            self.__dict__.update(page)
            self.draw_canvas(page_count)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)
    def draw_canvas(self, page_count):
        page = "Page %s of %s" % (self._pageNumber, page_count)        
        note2_1 = "The above Total Negative News Score is the a percentage between 0-100 with higher % representing higher risk."
        note1_1 = "The above Search Clause searches the input first name and last name with up to three word in between. The"
        note1_2 = "included quotes ensures that the searched name will appear in every result. In addition, at least one term from the above negative news " 
        note1_3 = "list will appear in every result analyzed. "
        note3_1 = "Links are filtered out by a pre-determined blacklist which includes, for example, links from LinkedIn or Facebook. "
        note4_1 = "Note that the above Name score will always be high due to the structure of the search clause. The confidence score for location"
        note4_2 = "is the result of an approximate or partial match between the input location information and the location that was found on the google "
        note4_3 = "search results. N/A implies location / Employment was not explicitly provided. "
        x = 128
        self.saveState()
        self.setStrokeColorRGB(0, 0, 0)
        self.setLineWidth(0.5)
        self.line(66, 150, letter[0] - 66, 150)
        self.setFont('Helvetica', 5)
        self.drawString(66, 140, '1')
        self.drawString(66, 130, '2')
        self.drawString(66, 120, '3')
        self.drawString(66, 90, '4')           
        self.setFont('Helvetica', 8)
        self.drawString(69, 140, note2_1)
        self.drawString(69, 130, note3_1)
        self.drawString(69, 120, note4_1)
        self.drawString(69, 110, note4_2)
        self.drawString(69, 100, note4_3)
        self.drawString(69, 90, note1_1)
        self.drawString(69, 80, note1_2)
        self.drawString(69, 70, note1_3)     
        self.drawString(letter[0]-x, 65, page)
        self.restoreState()

def add_hyperlink(paragraph, url, text, color, underline):
    """
    A function that places a hyperlink within a paragraph object.

    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Add color if it is given
    if not color is None:
        c = docx.oxml.shared.OxmlElement('w:color')
        c.set(docx.oxml.shared.qn('w:val'), color)
        rPr.append(c)

    # Remove underlining if it is requested
    if not underline:
        u = docx.oxml.shared.OxmlElement('w:u')
        u.set(docx.oxml.shared.qn('w:val'), 'none')
        rPr.append(u)

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink